##### Define Importdata

In [ ]:
start = "2024-02-29 22:00"
stop = "2025-03-31 22:00"
airport_str='LSZH'

##### Import of required libraries

In [ ]:
import pandas as pd
import numpy as np
import datetime
import requests
from traffic.data import opensky
from shapely.geometry import Polygon, Point
from traffic.core import Traffic
from traffic.data import airports
from traffic.data.weather import metar
from traffic.data import aircraft
from astral import LocationInfo
from astral.sun import sun
from datetime import timedelta
from matplotlib import pyplot as plt
from scipy.stats import norm
from scipy.spatial import cKDTree
from geopy.distance import geodesic  # For accurate distance calculation
from collections import Counter

In [ ]:
pd.set_option("display.max_columns", None)

##### Creating an empty Table with the required columns

In [ ]:
table = pd.DataFrame(columns=[
    'Event',
    'Runway',
    'Flight (Callsign)',
    'Flight ID',
    'ICAO Code', # Airline
    'A/C Type',
    'ICAO 24',
    'ICAO Aircraft Type',
    'Propulsion Type',
    'Number of Engines',
    'MALW [kg]',
    'ICAO Weight Turbulence Category',
    'Entry Time',
    'Exit Time',
    'Time of Go-around',
    'Time from Previous Landing [s]',
    'Time from Previous Event [s]',
    'Time to Next Event [s]',
    'Go-around',
    'Not Vacated',
    'Short ROT',
    'Gate Region',
    'RET',
    'ROT [s]'])
table

In [ ]:
columns = table.columns
columns

##### Creating Runway Polygons for all Runways in LSZH

##### Arrivals

##### Runway 14

In [ ]:
# Coordinates of Runway edges
runway_coordinates_14 = [
    (47.482343, 8.536430),
    (47.481882, 8.535696),
    (47.461058, 8.564121),
    (47.461527, 8.564832)
]

#Creating rectangle as polygon
runway_polygon_14 = Polygon(runway_coordinates_14)

##### Runway 16

In [ ]:
# Coordinates of Runway edges
runway_coordinates_16 = [
    (47.47585447393498, 8.536401636302713),
    (47.475571406110106, 8.535504664094239),
    (47.445370026120706, 8.556302195884056),
    (47.44564496451575, 8.55719874474424) 
]

#Creating rectangle as polygon
runway_polygon_16 = Polygon(runway_coordinates_16)

##### Runway 28

In [ ]:
# Coordinates of Runway edges
runway_coordinates_28 = [
    (47.456870896592605, 8.57050137471156),
    (47.45923408397896, 8.537529522304963),
    (47.45865425407874, 8.537438192760876),
    (47.456327096910435, 8.570421301131713)
]

#Creating rectangle as polygon
runway_polygon_28 = Polygon(runway_coordinates_28)

##### Runway 34

In [ ]:
# Coordinates of Runway edges
runway_coordinates_34 = [
    (47.47585447393498, 8.536401636302713),
    (47.475571406110106, 8.535504664094239),
    (47.44914103648339, 8.553699938937916),
    (47.44940877231636, 8.554611379237143)
]

#Creating rectangle as polygon
runway_polygon_34 = Polygon(runway_coordinates_34)

##### departures

##### Runway 10

In [ ]:
# Coordinates of Runway edges
runway_coordinates_10 = [
    (47.456870896592605, 8.57050137471156),
    (47.45923408397896, 8.537529522304963),
    (47.45865425407874, 8.537438192760876),
    (47.456327096910435, 8.570421301131713)
]

#Creating rectangle as polygon
runway_polygon_10 = Polygon(runway_coordinates_10)

##### Runway 32

In [ ]:
# Coordinates of Runway edges
runway_coordinates_32 = [
    (47.482343, 8.536430),
    (47.481882, 8.535696),
    (47.461058, 8.564121),
    (47.461527, 8.564832)
]

#Creating rectangle as polygon
runway_polygon_32 = Polygon(runway_coordinates_32)

##### Runway 34

In [ ]:
# Coordinates of Runway edges
runway_coordinates_dep_34 = [
    (47.47585447393498, 8.536401636302713),
    (47.475571406110106, 8.535504664094239),
    (47.445370026120706, 8.556302195884056),
    (47.44564496451575, 8.55719874474424)
]

#Creating rectangle as polygon
runway_polygon_dep_34 = Polygon(runway_coordinates_dep_34)

##### Creating polygons for the vacated area of all Runways at LSZH

##### Runway 10

In [ ]:
# Coordinates of Vacated Polygon edges
vacated_coordinates_10 = [
    (47.45776460288583, 8.57056862305337),
    (47.46008767461235, 8.537712376143574),
    (47.45776222821285, 8.53732795301875),
    (47.45555693585441, 8.570273050449208),
]


#Creating rectangle as polygon
runway_vacated_polygon_10 = Polygon(vacated_coordinates_10)

##### Runway 14

In [ ]:
# Coordinates of Vacated Polygon edges
vacated_coordinates_14 = [
    (47.482383375839085, 8.536704667668287),
    (47.48091837115578, 8.53426271448043),
    (47.45869688991013, 8.563109678080252),
    (47.46104054317113, 8.567008077240228),
]


#Creating rectangle as polygon
runway_vacated_polygon_14 = Polygon(vacated_coordinates_14)

##### Runway 16

In [ ]:
# Coordinates of Vacated Polygon edges
vacated_coordinates_16 = [
    (47.47740339814996, 8.538037143680414),
    (47.4763940907564, 8.533029582414327),
    (47.44477424019839, 8.55412239230834),
    (47.44625275590675, 8.559161932975567),
]


#Creating rectangle as polygon
runway_vacated_polygon_16 = Polygon(vacated_coordinates_16)

##### Runway 28

In [ ]:
# Coordinates of Vacated Polygon edges
vacated_coordinates_28 = [
    (47.45776460288583, 8.57056862305337),
    (47.46008767461235, 8.537712376143574),
    (47.45776222821285, 8.53732795301875),
    (47.45555693585441, 8.570273050449208),
]


#Creating rectangle as polygon
runway_vacated_polygon_28 = Polygon(vacated_coordinates_28)

##### Runway 32

In [ ]:
# Coordinates of Vacated Polygon edges
vacated_coordinates_32 = [
    (47.482383375839085, 8.536704667668287),
    (47.48091837115578, 8.53426271448043),
    (47.45869688991013, 8.563109678080252),
    (47.46104054317113, 8.567008077240228),
]


#Creating rectangle as polygon
runway_vacated_polygon_32 = Polygon(vacated_coordinates_32)

##### Runway 34

In [ ]:
# Coordinates of Runway edges
vacated_coordinates_34 = [
    (47.47740339814996, 8.538037143680414),
    (47.4763940907564, 8.533029582414327),
    (47.44477424019839, 8.55412239230834),
    (47.44625275590675, 8.559161932975567),
]


#Creating rectangle as polygon
runway_vacated_polygon_34 = Polygon(vacated_coordinates_34)

##### Creating Runway End Polygons for all Runway at LSZH

##### Runway 10

In [ ]:
# Coordinates of Go-Around Polygon edges
rwy_end_coordinates_10 = [
    (47.46148751731977, 8.59971679761919),
    (47.460227088786795, 8.572533343052678),
    (47.45411938930318, 8.571529657185232),
    (47.44874611374969, 8.597072087487838),
]


#Creating rectangle as polygon
runway_end_polygon_10 = Polygon(rwy_end_coordinates_10)

##### Runway 14

In [ ]:
# Coordinates of Go-Around Polygon edges
rwy_end_coordinates_14 = [
    (47.461967, 8.566471),
    (47.460347, 8.563795),
    (47.458375, 8.566194),
    (47.460188, 8.568865),
]


#Creating rectangle as polygon
runway_end_polygon_14 = Polygon(rwy_end_coordinates_14)

##### Runway 16

In [ ]:
# Coordinates of Go-Around Polygon edges
rwy_end_coordinates_16 = [
    (47.44520141702879, 8.558613297414626),
    (47.44461364335362, 8.555759426776678),
    (47.423287251388544, 8.56526483194869),
    (47.42753821549068, 8.615097914353424),
]


#Creating rectangle as polygon
runway_end_polygon_16 = Polygon(rwy_end_coordinates_16)

##### Runway 28

In [ ]:
# Coordinates of Go-Around Polygon edges
rwy_end_coordinates_28 = [
    (47.46292957458374, 8.532905101022967),
    (47.47354502175972, 8.469275023084766),
    (47.44935426660808, 8.467238908246937),
    (47.4521457251421, 8.529301410645639),
]


#Creating rectangle as polygon
runway_end_polygon_28 = Polygon(rwy_end_coordinates_28)

##### Runway 32

In [ ]:
# Coordinates of Go-Around Polygon edges
rwy_end_coordinates_32 = [
    (47.51206078888496, 8.52465454416451),
    (47.501827520767094, 8.492696049234075),
    (47.48173484357998, 8.531900549857157),
    (47.486370608636484, 8.537272617311466),
]


#Creating rectangle as polygon
runway_end_polygon_32 = Polygon(rwy_end_coordinates_32)

##### Runway 34

In [ ]:
# Coordinates of Go-Around Polygon edges
rwy_end_coordinates_34 = [
    (47.50101544619973, 8.530959232219672),
    (47.49890923290986, 8.512478211967327),
    (47.476456999467196, 8.533220648187603),
    (47.47721875920997, 8.53682202609175),
]


#Creating rectangle as polygon
runway_end_polygon_34 = Polygon(rwy_end_coordinates_34)

##### Creating departure End Polygons

In [ ]:
# Coordinates of Go-Around Polygon edges
extra_rwy_end_coordinates_28 = [
    (47.45249632457016, 8.553325972675136),
    (47.45506244093026, 8.545116396099612),
    (47.44469246672567, 8.530091321989694),
    (47.436258808740114, 8.54898883788052),
]


#Creating rectangle as polygon
extra_runway_end_polygon_28 = Polygon(extra_rwy_end_coordinates_28)

##### RET Polygons

In [ ]:
# Coordinates of RET
G_coordinates = [
    (47.47900496089672, 8.539611392549546),
    (47.47792619841368, 8.537963434384269),
    (47.47769562341817, 8.538209379739097),
    (47.47874602257377, 8.539969300376018),
]


#Creating rectangle as polygon
G = Polygon(G_coordinates)

In [ ]:
# Coordinates of RET
H1_coordinates = [
    (47.466040, 8.557257),
    (47.467266, 8.555648),
    (47.464178, 8.556998),
    (47.464282, 8.557709),
]


#Creating rectangle as polygon
H1 = Polygon(H1_coordinates)

In [ ]:
# Coordinates of RET
H2_coordinates = [
    (47.464202, 8.559839),
    (47.465221, 8.558396),
    (47.462913, 8.559442),
    (47.462692, 8.560185),
]


#Creating rectangle as polygon
H2 = Polygon(H2_coordinates)

In [ ]:
# Coordinates of RET
H3_coordinates = [
    (47.460924, 8.564330),
    (47.461881, 8.562975),
    (47.460752, 8.562466),
    (47.459968, 8.563603),
]


#Creating rectangle as polygon
H3 = Polygon(H3_coordinates)

In [ ]:
# Coordinates of RET
B1_coordinates = [
    (47.45756545754382, 8.570671828471193),
    (47.45777872852484, 8.569933805965368),
    (47.45693270072414, 8.569837471054846),
    (47.45687294107219, 8.570501844387083),
]


#Creating rectangle as polygon
B1 = Polygon(B1_coordinates)

In [ ]:
# Coordinates of RET
A1_coordinates = [
    (47.45632934014095, 8.570418239562354),
    (47.45637948657381, 8.569570102208095),
    (47.45577451166752, 8.569449347226142),
    (47.455724364657705, 8.570725356564049)
]


#Creating rectangle as polygon
A1 = Polygon(A1_coordinates)

In [ ]:
# Coordinates of RET
KN_coordinates = [
    (47.45792085293085, 8.563414129207736),
    (47.45800644045029, 8.562283037806466),
    (47.45747082593954, 8.562283037806466),
    (47.45741836855496, 8.563214044447221),
]


#Creating rectangle as polygon
KN = Polygon(KN_coordinates)

In [ ]:
# Coordinates of RET
KS_coordinates = [
    (47.4568330510224, 8.563175252503983),
    (47.45691035748411, 8.562195245513713),
    (47.45635264403403, 8.56215032852666),
    (47.456289141632425, 8.563052751630199),
]


#Creating rectangle as polygon
KS = Polygon(KS_coordinates)

In [ ]:
# Coordinates of RET
JN_coordinates = [
    (47.45818601284084, 8.559378717169675),
    (47.45828108091835, 8.55826684236473),
    (47.45776332345507, 8.558223578754731),
    (47.45769458126239, 8.559216478629185),
]


#Creating rectangle as polygon
JN = Polygon(JN_coordinates)

In [ ]:
# Coordinates of RET
JS_coordinates = [
    (47.457122005384264, 8.559137967367773),
    (47.457190170145765, 8.558148896668472),
    (47.45663379572564, 8.558026284598311),
    (47.45654720712422, 8.55918973690851),
]


#Creating rectangle as polygon
JS = Polygon(JS_coordinates)

In [ ]:
# Coordinates of RET
FN_coordinates = [
    (47.45867085296834, 8.552643892559512),
    (47.45879463622604, 8.551488781761993),
    (47.458206200754915, 8.551988186099704),
    (47.458147578906804, 8.552950544345826),
]


#Creating rectangle as polygon
FN = Polygon(FN_coordinates)

In [ ]:
FS_coordinates = [
    (47.45750512236538, 8.553746541065578),
    (47.45760213071381, 8.552357287372269),
    (47.45702007793867, 8.552493763175892),
    (47.456956193705096, 8.553554075188673),
]


#Creating rectangle as polygon
FS = Polygon(FS_coordinates)

In [ ]:
# Coordinates of RET
EN_coordinates = [
    (47.45876640251088, 8.551198492745044),
    (47.45890275119224, 8.549920151173053),
    (47.458315756048464, 8.550415764884118),
    (47.458246425480624, 8.551441172562187),
]


#Creating rectangle as polygon
EN = Polygon(EN_coordinates)

In [ ]:
# Coordinates of RET
ES_coordinates = [
    (47.45760131253341, 8.551966860047404),
    (47.45769162537079, 8.550921854075147),
    (47.45694786797097, 8.551393285340826),  
    (47.45709396400481, 8.551982574422928),  
]


#Creating rectangle as polygon
ES = Polygon(ES_coordinates)

In [ ]:
# Coordinates of RET
RET34_coordinates = [
    (47.459492016042034, 8.547668491560723),  
    (47.45948305465726, 8.546578325129305),  
    (47.45854210075558, 8.547231099557843),  
    (47.45843008131226, 8.548821615931946),  
]


#Creating rectangle as polygon
RET34 = Polygon(RET34_coordinates)

In [ ]:
# Coordinates of RET
RET16_coordinates = [
    (47.45779036018414, 8.549572124749611),  
    (47.457931801108764, 8.547629182176722),  
    (47.45681337468931, 8.548404644389631),  
    (47.4569133685967, 8.549450425507892),  
]


#Creating rectangle as polygon
RET16 = Polygon(RET16_coordinates)

In [ ]:
# Coordinates of RET
B7_coordinates = [
    (47.459428835765074, 8.543656448936817),  
    (47.45954906355491, 8.542339878695742),  
    (47.458883183109705, 8.542391173640201),  
    (47.45879532325455, 8.54381033377019),  
]


#Creating rectangle as polygon
B7 = Polygon(B7_coordinates)

In [ ]:
# Coordinates of RET
L7_coordinates = [
    (47.45802487489487, 8.546156075560166),  
    (47.45819483993097, 8.544086259000071),  
    (47.457483958741975, 8.541936859026693),  
    (47.45726276579127, 8.542510418945927),  
]


#Creating rectangle as polygon
L7 = Polygon(L7_coordinates)

In [ ]:
# Coordinates of RET
B9_coordinates = [
    (47.459695755684216, 8.538385030745486),  
    (47.4599965199352, 8.537735601278188),  
    (47.459225305469744, 8.537602384975905),  
    (47.4591899210105, 8.538182827430195),  
]


#Creating rectangle as polygon
B9 = Polygon(B9_coordinates)

In [ ]:
# Coordinates of RET
L9_coordinates = [
    (47.458589114605275, 8.538425914498577),  
    (47.458667926314384, 8.53733401660301),  
    (47.45769649425433, 8.537243800999553),  
    (47.457671539534516, 8.537918209361699),  
]


#Creating rectangle as polygon
L9 = Polygon(L9_coordinates)

In [ ]:
# Coordinates of RET
E1_coordinates = [
    (47.476784711708326, 8.537128209888078),  
    (47.47703506093943, 8.536748549001688),  
    (47.47573522165197, 8.53599970768071),  
    (47.47585424648102, 8.536393301610623),  
]


#Creating rectangle as polygon
E1 = Polygon(E1_coordinates)

In [ ]:
# Coordinates of RET
E2_coordinates = [
    (47.47623843379217, 8.53759186425323),  
    (47.4758454943537, 8.536401903798348),  
    (47.47537301810616, 8.536737258990241),  
    (47.47594176110893, 8.537664848352282),  
]


#Creating rectangle as polygon
E2 = Polygon(E2_coordinates)

In [ ]:
# Coordinates of RET
E3_coordinates = [
    (47.47039445040321, 8.541293882494458),  
    (47.47021748472535, 8.540273328418696),  
    (47.46971057973998, 8.540606117791228),  
    (47.4700765116403, 8.541462495776543),  
]


#Creating rectangle as polygon
E3 = Polygon(E3_coordinates)

In [ ]:
# Coordinates of RET
E4_coordinates = [
    (47.46489160961231, 8.545610163012801),  
    (47.46446586487309, 8.5442543697524216),  
    (47.46404507297492, 8.54454043537966),  
    (47.46462575303669, 8.545677361848597),  
]


#Creating rectangle as polygon
E4 = Polygon(E4_coordinates)

In [ ]:
# Coordinates of RET
E5_coordinates = [
    (47.46049003012993, 8.548353272951783),  
    (47.46200814316435, 8.545934371171334),  
    (47.4605966236558, 8.546897304112772),  
    (47.459925078346075, 8.548543511406802),  
]


#Creating rectangle as polygon
E5 = Polygon(E5_coordinates)

In [ ]:
# Coordinates of RET
BW_coordinates = [
    (47.45948235726516, 8.546575349523982),  
    (47.45956998885824, 8.545191546915806),  
    (47.459339687121265, 8.545153948884279),  
    (47.459234505344924, 8.546748522132432),  
]


#Creating rectangle as polygon
BW = Polygon(BW_coordinates)

In [ ]:
# Coordinates of RET
BO_coordinates = [
    (47.45938683316601, 8.54919315902424),  
    (47.45944831772536, 8.547702942829474),  
    (47.45895346414919, 8.548047628662957),  
    (47.459020899379205, 8.549367702063828),  
]


#Creating rectangle as polygon
BO = Polygon(BO_coordinates)

In [ ]:
# Coordinates of RET
RET28_coordinates = [
    (47.458562221857086, 8.547191778865505),  
    (47.45866140593849, 8.545606013797736),  
    (47.45805363571239, 8.545606013797736),  
    (47.45793997060175, 8.547646775950202),  
]


#Creating rectangle as polygon
RET28 = Polygon(RET28_coordinates)

In [ ]:
# Coordinates of RET
RET10_coordinates = [
    (47.458312751246474, 8.550465977237026),  
    (47.45846872752979, 8.548365787332475),  
    (47.45748150203258, 8.549057748271517),  
    (47.45773066266478, 8.550528779990797),  
]


#Creating rectangle as polygon
RET10 = Polygon(RET10_coordinates)

In [ ]:
# Coordinates of RET
E6_coordinates = [
    (47.456346509461966, 8.551301052255788),  
    (47.45517659975349, 8.550639813446152),  
    (47.45474437446953, 8.550927608128692),  
    (47.455939806004075, 8.551740855044246),  
]


#Creating rectangle as polygon
E6 = Polygon(E6_coordinates)

In [ ]:
# Coordinates of RET
L34_coordinates = [
    (47.45472861497266, 8.549837677202353),  
    (47.455034051389426, 8.54857080903866),  
    (47.45469349087491, 8.548415107557833),  
    (47.45429146181007, 8.550144865573472),  
]


#Creating rectangle as polygon
L34 = Polygon(L34_coordinates)

In [ ]:
# Coordinates of RET
E7_coordinates = [
    (47.45391983044209, 8.55324495783399),  
    (47.454633741996005, 8.551001415766288),  
    (47.453765707022264, 8.551602890014872),  
    (47.45334892028282, 8.553525858979743),  
]


#Creating rectangle as polygon
E7 = Polygon(E7_coordinates)

In [ ]:
# Coordinates of RET
R7_coordinates = [
    (47.454171001301866, 8.550233814044592),  
    (47.45294462600691, 8.549678783107847),  
    (47.452774828019734, 8.550229287866989),  
    (47.45363687366552, 8.55061560699621),  
]


#Creating rectangle as polygon
R7 = Polygon(R7_coordinates)

In [ ]:
# Coordinates of RET
E8_coordinates = [
    (47.44955894047596, 8.55589207216645),  
    (47.44926025881387, 8.554724298716822),  
    (47.448886904351006, 8.554971906840812),  
    (47.44928967450742, 8.55597572355969),  
]


#Creating rectangle as polygon
E8 = Polygon(E8_coordinates)

In [ ]:
# Coordinates of RET
R8_coordinates = [
    (47.44900680609096, 8.553790384315583),  
    (47.448588194344346, 8.55284679659984),  
    (47.44835739076729, 8.55298733094048),  
    (47.44856104103492, 8.554165142557297),  
]


#Creating rectangle as polygon
R8 = Polygon(R8_coordinates)

In [ ]:
# Coordinates of RET
E9_coordinates = [
    (47.446286251798966, 8.557804793264934),  
    (47.44636484072531, 8.556591699571102),  
    (47.445623033256204, 8.557124553810448),  
    (47.445929724951164, 8.558085392039908),  
]


#Creating rectangle as polygon
E9 = Polygon(E9_coordinates)

##### RET Dictionaries for every single runway at LSZH

In [ ]:
# Define polygons  
ret_dict_10 = {"A1": A1, "B1": B1, "KN": KN, "KS": KS, "JN": JN, "JS": JS, "FN": FN, "FS": FS, "EN": EN, "ES": ES, "34": RET34, "16": RET16, "B7": B7, "L7": L7, "B9": B9, "L9": L9}
ret_dict_14 = {"G": G, "H1": H1, "H2": H2, "H3": H3}
ret_dict_16 = {"E1": E1, "E2": E2, "E3": E3, "E4": E4, "E5": E5, "BW": BW, "BO": BO, "28": RET28, "10": RET10, "E6": E6, "L": L34, "E7": E7, "R7": R7, "E8": E8, "R8": R8, "E9": E9}
ret_dict_28 = {"A1": A1, "B1": B1, "KN": KN, "KS": KS, "JN": JN, "JS": JS, "FN": FN, "FS": FS, "EN": EN, "ES": ES, "34": RET34, "16": RET16, "B7": B7, "L7": L7, "B9": B9, "L9": L9}
ret_dict_32 = {"G": G, "H1": H1, "H2": H2, "H3": H3}
ret_dict_34 = {"E1": E1, "E2": E2, "E3": E3, "E4": E4, "E5": E5, "BW": BW, "BO": BO, "28": RET28, "10": RET10, "E6": E6, "L": L34, "E7": E7, "R7": R7, "E8": E8, "R8": R8, "E9": E9}

##### Function to determine in which RET a Point lies

In [ ]:
# Function to determine which polygon a point belongs to  
def find_polygon(lat, lon, ret_dict):
    point = Point(lat, lon)  # Shapely expects (longitude, latitude) as (x, y)
    for name, polygon in ret_dict.items():
        if polygon.contains(point) or polygon.boundary.contains(point):  # Check if point is inside or on the boundary
            return name
    return "Outside"  # Return "Outside" if the point is not in any polygon


##### Define Import Geometry Bounds

In [ ]:
# Coordinates of Import Polygon
import_coordinates = [
    (9.3629975008778, 47.572540686883436),  
    (8.11652924277223, 48.039507633659085),  
    (7.467456560886543, 47.47109868425854),
    (8.445791459475736, 47.29062332727335),  
]

#Creating rectangle as polygon
Import_Polygon = Polygon(import_coordinates)

#### Gate Region Coordinates

In [ ]:
# Coordinates of gate edges
gate_coordinates_North = [
    (47.46222690988436, 8.559511063245429), #RWY 14 right
    (47.4677137606189, 8.545262706862454), #RWY 14 left
    (47.46039783236904, 8.550989103256876), #RWY 32 left
    (47.459544241084316, 8.559691422186987)  #RWY 32 right
]
 
#Creating rectangle as polygon
gate_polygon_North = Polygon(gate_coordinates_North)
 
# Coordinates of gate edges
gate_coordinates_East = [
    (47.45860892612085, 8.576476605347214), #RWY 14 right
    (47.4595824620173, 8.565050900074354), #RWY 14 left
    (47.454039744220424, 8.56344260504518), #RWY 32 left
    (47.453085179118666, 8.576320702453943)  #RWY 32 right
]
 
#Creating rectangle as polygon
gate_polygon_East = Polygon(gate_coordinates_East)
 
# Coordinates of gate edges
gate_coordinates_South = [
    (47.45596248073526, 8.56432081618345), #RWY 14 right
    (47.457155456631696, 8.550783852563612), #RWY 14 left
    (47.43975178123579, 8.560427038086573), #RWY 32 left
    (47.44306209954342, 8.570948773913384)  #RWY 32 right
]
 
#Creating rectangle as polygon
gate_polygon_South = Polygon(gate_coordinates_South)
 
# Coordinates of gate edges
gate_coordinates_West = [
    (47.457906230675164, 8.547819449164445), #RWY 14 right
    (47.458555814138954, 8.537937541532848), #RWY 14 left
    (47.443891833614515, 8.54946643376971), #RWY 32 left
    (47.445562620043646, 8.55619162090788)  #RWY 32 right
]
 
#Creating rectangle as polygon
gate_polygon_West = Polygon(gate_coordinates_West)

# Gate polygons already defined
gate_polygons = {
    "North": gate_polygon_North,
    "East": gate_polygon_East,
    "South": gate_polygon_South,
    "West": gate_polygon_West
}

##### Import Flightdata and Import Flightlist

In [ ]:
timestamps = pd.date_range(start, stop, freq="24h")
data = []
for t1, t2 in zip(timestamps[:-1], timestamps[1:]):
        print(t1, t2)
        flight_list = opensky.flightlist(start=t1, stop=t2, arrival_airport=airport_str)
        icao24 = flight_list.icao24
        tmp = opensky.history(start=str(t1), stop=str(t2), bounds=Import_Polygon, icao24=icao24, return_flight=True)
        if tmp is not None:
            data.append(tmp.data)
        
 
trajs = pd.concat(data)
trajs = Traffic(trajs)

In [ ]:
def convert_timestamp(x):
    if isinstance(x, (float, int)) and x > 1e9:  # Convert to unix timestamps if float or int.
        return pd.to_datetime(x, unit='s', errors='coerce')
    elif isinstance(x, pd.Timestamp): # If x is alreardy a pandas timestamp, x will be returned
        return x
    else:
        return pd.NaT  # Invalid Value will be converted to Not a Timestamp

# Make sure data format is "correct"
trajs.data['timestamp'] = trajs.data['timestamp'].apply(convert_timestamp)
trajs.data['hour'] = trajs.data['hour'].apply(convert_timestamp)
trajs.data['onground'] = trajs.data['onground'].astype(np.bool_)
trajs.data['alert'] = trajs.data['alert'].astype(np.bool_)
trajs.data['spi'] = trajs.data['spi'].astype(np.bool_)
trajs.data['last_position'] = trajs.data['last_position'].astype(np.float64)

trajs.data['latitude'] = trajs.data['latitude'].astype('double')
trajs.data['longitude'] = trajs.data['longitude'].astype('double')
trajs.data['groundspeed'] = trajs.data['groundspeed'].astype('double')
trajs.data['track'] = trajs.data['track'].astype('double')
trajs.data['vertical_rate'] = trajs.data['vertical_rate'].astype('double')
trajs.data['altitude'] = trajs.data['altitude'].astype('double')
trajs.data['geoaltitude'] = trajs.data['geoaltitude'].astype('double')
trajs.data['lastcontact'] = trajs.data['lastcontact'].astype('double')

trajs.data['icao24'] = trajs.data['icao24'].astype('object')
trajs.data['callsign'] = trajs.data['callsign'].astype('object')
trajs.data['squawk'] = trajs.data['squawk'].astype('object')

##### Assign ID

In [ ]:
trajs = trajs.assign_id().eval(max_workers=1, desc='Assign Flight ID')

##### Drop duplicates

In [ ]:
trajs.data = trajs.data.drop_duplicates(
    subset=["flight_id", "last_position"])

##### Remove all data above 8000ft to get only relevant datapoints

In [ ]:
trajs_relevant = trajs.query('geoaltitude < 8000 or onground or geoaltitude.isna()')

##### Resample Data

In [ ]:
trajs_relevant = trajs_relevant.resample('1s').eval(max_workers=40, desc='Resample Flights')

##### Get a List of all Flight IDs

In [ ]:
all_flight_ids = trajs_relevant.flight_ids

#### Get a List with all Go Arounds

In [ ]:
def has_go_around(flight):
    if flight.go_around('LSZH').has() == True:
        return flight
    
go_around = trajs_relevant.iterate_lazy().pipe(has_go_around).eval(max_workers=40, desc='Processing')
if go_around:
    go_around_ids = go_around.flight_ids

#### Filter all departure flights

In [ ]:
def departure_rwy(flight):
    x = flight.takeoff('LSZH', method="track_based")

    if x is not None:
        x_next = x.next()
        if x_next is not None:
            rwy = x_next.max('runway')
            if rwy == '14':
                track = x_next.data['track'].median()
                if track >= 145:
                    return {
                        'Flight ID': flight.flight_id,
                        'Runway': '16'
                    }
            else:
                return {
                    'Flight ID': flight.flight_id,
                    'Runway': rwy,
        }
        
departures = trajs.iterate_lazy().pipe(departure_rwy).eval(max_workers=40, desc= 'Processing Flights')

#### Remove Go Arounds from departures and create trajs_remaining

In [ ]:
if 'go_around_ids' in locals():
    if go_around_ids:
        departures = departures[~departures['Flight ID'].isin(go_around_ids)]
departures_ids = departures['Flight ID'].unique()

#### Filter all Arrivals

In [ ]:
def arrival_rwy(flight):
    x = flight.aligned_on_ils('LSZH')
    if x is not None:
        x_next = x.max()
        if x_next is not None:
            rwy = x_next.max('ILS')
            on_ground = x_next.data['onground'].median()
            if on_ground == 0:
                vertical_rate = x_next.data['vertical_rate'].median()
                if vertical_rate < -200:
                    return {
                        'Flight ID': flight.flight_id,
                        'Runway': rwy,
                        'Vertical Rate': vertical_rate
    }
        
arrivals = trajs.iterate_lazy().pipe(arrival_rwy).eval(max_workers=40, desc= 'Processing Flights')

#### Create trajs for each departure runway

In [ ]:
flight_ids_dep_10 = departures.loc[departures["Runway"] == "10", "Flight ID"].tolist()
flight_ids_dep_16 = departures.loc[departures["Runway"] == "16", "Flight ID"].tolist()
flight_ids_dep_28 = departures.loc[departures["Runway"] == "28", "Flight ID"].tolist()
flight_ids_dep_32 = departures.loc[departures["Runway"] == "32", "Flight ID"].tolist()
flight_ids_dep_34 = departures.loc[departures["Runway"] == "34", "Flight ID"].tolist()

In [ ]:
trajs_dep_10 = trajs_relevant[flight_ids_dep_10]
trajs_dep_16 = trajs_relevant[flight_ids_dep_16]
trajs_dep_28 = trajs_relevant[flight_ids_dep_28]
trajs_dep_32 = trajs_relevant[flight_ids_dep_32]
trajs_dep_34 = trajs_relevant[flight_ids_dep_34]

#### Create trajs for each arrival runway

In [ ]:
flight_ids_arr_14 = arrivals.loc[arrivals["Runway"] == "14", "Flight ID"].tolist()
flight_ids_arr_16 = arrivals.loc[arrivals["Runway"] == "16", "Flight ID"].tolist()
flight_ids_arr_28 = arrivals.loc[arrivals["Runway"] == "28", "Flight ID"].tolist()
flight_ids_arr_34 = arrivals.loc[arrivals["Runway"] == "34", "Flight ID"].tolist()

In [ ]:
trajs_arr_14 = trajs_relevant[flight_ids_arr_14]
trajs_arr_16 = trajs_relevant[flight_ids_arr_16]
trajs_arr_28 = trajs_relevant[flight_ids_arr_28]
trajs_arr_34 = trajs_relevant[flight_ids_arr_34]

##### Filter data just for points which are within the polygon of the corresponding runway

##### departures Runway 10

In [ ]:
# Function that checks if a point is within one of the polygons
def point_in_polygons(row):
    point = Point(row['latitude'], row['longitude'])
    in_runway = runway_polygon_10.contains(point) or runway_polygon_10.intersects(point)
    in_vacated = runway_vacated_polygon_10.contains(point) or runway_vacated_polygon_10.intersects(point)
    in_end = runway_end_polygon_10.contains(point) or runway_end_polygon_10.intersects(point)
    return in_runway or in_vacated or in_end

def flight_in_polygons(flight):
    flight.data = flight.data.loc[flight.data.apply(point_in_polygons, axis=1)]
    return flight


# Filter the DataFrame using the apply method with lazy execution
trajs_dep_10 = trajs_dep_10.iterate_lazy().pipe(flight_in_polygons).eval(max_workers=40, desc= 'Processing')

##### departures Runway 16

In [ ]:
# Function that checks if a point is within one of the polygons
def point_in_polygons(row):
    point = Point(row['latitude'], row['longitude'])
    in_runway = runway_polygon_16.contains(point) or runway_polygon_16.intersects(point)
    in_vacated = runway_vacated_polygon_16.contains(point) or runway_vacated_polygon_16.intersects(point)
    in_end = runway_end_polygon_16.contains(point) or runway_end_polygon_16.intersects(point)
    return in_runway or in_vacated or in_end

def flight_in_polygons(flight):
    flight.data = flight.data.loc[flight.data.apply(point_in_polygons, axis=1)]
    return flight


# Filter the DataFrame using the apply method with lazy execution
trajs_dep_16 = trajs_dep_16.iterate_lazy().pipe(flight_in_polygons).eval(max_workers=40, desc= 'Processing')

##### departures Runway 28

In [ ]:
# Function that checks if a point is within one of the polygons
def point_in_polygons(row):
    point = Point(row['latitude'], row['longitude'])
    in_runway = runway_polygon_28.contains(point) or runway_polygon_28.intersects(point)
    in_vacated = runway_vacated_polygon_28.contains(point) or runway_vacated_polygon_28.intersects(point)
    in_end = runway_end_polygon_28.contains(point) or runway_end_polygon_28.intersects(point)
    in_extra = extra_runway_end_polygon_28.contains(point) or extra_runway_end_polygon_28.intersects(point)
    return in_runway or in_vacated or in_end or in_extra

def flight_in_polygons(flight):
    flight.data = flight.data.loc[flight.data.apply(point_in_polygons, axis=1)]
    return flight


# Filter the DataFrame using the apply method with lazy execution
trajs_dep_28 = trajs_dep_28.iterate_lazy().pipe(flight_in_polygons).eval(max_workers=40, desc= 'Processing')

##### departures Runway 32

In [ ]:
# Function that checks if a point is within one of the polygons
def point_in_polygons(row):
    point = Point(row['latitude'], row['longitude'])
    in_runway = runway_polygon_32.contains(point) or runway_polygon_32.intersects(point)
    in_vacated = runway_vacated_polygon_32.contains(point) or runway_vacated_polygon_32.intersects(point)
    in_end = runway_end_polygon_32.contains(point) or runway_end_polygon_32.intersects(point)
    return in_runway or in_vacated or in_end

def flight_in_polygons(flight):
    flight.data = flight.data.loc[flight.data.apply(point_in_polygons, axis=1)]
    return flight


# Filter the DataFrame using the apply method with lazy execution
trajs_dep_32 = trajs_dep_32.iterate_lazy().pipe(flight_in_polygons).eval(max_workers=40, desc= 'Processing')

##### departures Runway 34

In [ ]:
# Function that checks if a point is within one of the polygons
def point_in_polygons(row):
    point = Point(row['latitude'], row['longitude'])
    in_runway = runway_polygon_dep_34.contains(point) or runway_polygon_dep_34.intersects(point)
    in_vacated = runway_vacated_polygon_34.contains(point) or runway_vacated_polygon_34.intersects(point)
    in_end = runway_end_polygon_34.contains(point) or runway_end_polygon_34.intersects(point)
    return in_runway or in_vacated or in_end

def flight_in_polygons(flight):
    flight.data = flight.data.loc[flight.data.apply(point_in_polygons, axis=1)]
    return flight


# Filter the DataFrame using the apply method with lazy execution
trajs_dep_34 = trajs_dep_34.iterate_lazy().pipe(flight_in_polygons).eval(max_workers=40, desc= 'Processing')

##### Arrival Runway 14

In [ ]:
# Function that checks if a point is within one of the polygons
def point_in_polygons(row):
    point = Point(row['latitude'], row['longitude'])
    in_runway = runway_polygon_14.contains(point) or runway_polygon_14.intersects(point)
    in_vacated = runway_vacated_polygon_14.contains(point) or runway_vacated_polygon_14.intersects(point)
    in_end = runway_end_polygon_14.contains(point) or runway_end_polygon_14.intersects(point)
    return in_runway or in_vacated or in_end

def flight_in_polygons(flight):
    flight.data = flight.data.loc[flight.data.apply(point_in_polygons, axis=1)]
    return flight


# Filter the DataFrame using the apply method with lazy execution
trajs_arr_14 = trajs_arr_14.iterate_lazy().pipe(flight_in_polygons).eval(max_workers=40, desc= 'Processing')

##### Arrival Runway 16

In [ ]:
# Function that checks if a point is within one of the polygons
def point_in_polygons(row):
    point = Point(row['latitude'], row['longitude'])
    in_runway = runway_polygon_16.contains(point) or runway_polygon_16.intersects(point)
    in_vacated = runway_vacated_polygon_16.contains(point) or runway_vacated_polygon_16.intersects(point)
    in_end = runway_end_polygon_16.contains(point) or runway_end_polygon_16.intersects(point)
    return in_runway or in_vacated or in_end

def flight_in_polygons(flight):
    flight.data = flight.data.loc[flight.data.apply(point_in_polygons, axis=1)]
    return flight


# Filter the DataFrame using the apply method with lazy execution
trajs_arr_16 = trajs_arr_16.iterate_lazy().pipe(flight_in_polygons).eval(max_workers=40, desc= 'Processing')

##### Arrival Runway 28

In [ ]:
# Function that checks if a point is within one of the polygons
def point_in_polygons(row):
    point = Point(row['latitude'], row['longitude'])
    in_runway = runway_polygon_28.contains(point) or runway_polygon_28.intersects(point)
    in_vacated = runway_vacated_polygon_28.contains(point) or runway_vacated_polygon_28.intersects(point)
    in_end = runway_end_polygon_28.contains(point) or runway_end_polygon_28.intersects(point)
    return in_runway or in_vacated or in_end

def flight_in_polygons(flight):
    flight.data = flight.data.loc[flight.data.apply(point_in_polygons, axis=1)]
    return flight


# Filter the DataFrame using the apply method with lazy execution
trajs_arr_28 = trajs_arr_28.iterate_lazy().pipe(flight_in_polygons).eval(max_workers=40, desc= 'Processing')

##### Arrival Runway 34

In [ ]:
# Function that checks if a point is within one of the polygons
def point_in_polygons(row):
    point = Point(row['latitude'], row['longitude'])
    in_runway = runway_polygon_34.contains(point) or runway_polygon_34.intersects(point)
    in_vacated = runway_vacated_polygon_34.contains(point) or runway_vacated_polygon_34.intersects(point)
    in_end = runway_end_polygon_34.contains(point) or runway_end_polygon_34.intersects(point)
    return in_runway or in_vacated or in_end

def flight_in_polygons(flight):
    flight.data = flight.data.loc[flight.data.apply(point_in_polygons, axis=1)]
    return flight


# Filter the DataFrame using the apply method with lazy execution
trajs_arr_34 = trajs_arr_34.iterate_lazy().pipe(flight_in_polygons).eval(max_workers=40, desc= 'Processing')

##### Define general Function to determine ROT of departure Flights and further features

In [ ]:
# Define the processing function for each departure flight
def process_flight_dep(flight, rwy, runway_polygon, entry_polygon, end_polygon, dict_ret,
                       altitude_threshold_runway, altitude_threshold_end, time_th, required_end_points, required_entry_points,
                       extra_end_polygon=None):

    # Initialize variables (do not change)
    points_inside = []
    points_entry = []
    points_indside_end = []
    end_points_count = 0
    last_valid_altitude = 0
    has_no_end_data = True
    timestamp_time = None

    flight = flight.sort_values(by='timestamp')

    # Iterate through flight data to collect position and altitude
    for _, row in flight.data.iterrows():
        lat = row['latitude']
        lon = row['longitude']
        alt = row['geoaltitude']

        if np.isnan(alt) and last_valid_altitude is not None:
            alt = last_valid_altitude

        point = Point(lat, lon)

        if (runway_polygon.contains(point) or runway_polygon.intersects(point)) and alt <= altitude_threshold_runway:
            points_inside.append(row)
            if not np.isnan(alt):
                last_valid_altitude = alt

        if ((entry_polygon.contains(point) or entry_polygon.intersects(point)) and not (runway_polygon.contains(point) or runway_polygon.intersects(point)) and not (points_inside)):
            points_entry.append(row)

        if extra_end_polygon is None:
            if (end_polygon.contains(point) or end_polygon.intersects(point)) and alt > altitude_threshold_end:
                points_indside_end.append(row)
                end_points_count += 1
        else:
            if (end_polygon.contains(point) or end_polygon.intersects(point) or extra_end_polygon.contains(point) or extra_end_polygon.intersects(point)) and alt > altitude_threshold_end:
                points_indside_end.append(row)
                end_points_count += 1

        if end_points_count >= required_end_points:
            break

    flight_data_rwy = pd.DataFrame(points_inside)
    flight_rwy_end_data = pd.DataFrame(points_indside_end)
    entry_points = pd.DataFrame(points_entry)

    if not flight_rwy_end_data.empty:
        has_no_end_data = False
        if not flight_data_rwy.empty:
            last_end_point_time = pd.to_datetime(flight_rwy_end_data['timestamp'].max())
            flight_data_rwy = flight_data_rwy[flight_data_rwy["timestamp"] <= last_end_point_time]

    if not flight_data_rwy.empty:
        first_timestamp = pd.to_datetime(flight_data_rwy['timestamp'].min())
        last_timestamp = pd.to_datetime(flight_data_rwy['timestamp'].max())
        rot = (last_timestamp - first_timestamp).total_seconds()

        flight_data_rwy = flight_data_rwy.sort_values(by='timestamp').reset_index(drop=True)
        flight_data_rwy['time_diff'] = flight_data_rwy['timestamp'].diff().dt.total_seconds()
        gap_index = flight_data_rwy[flight_data_rwy['time_diff'] > time_th].index
        flight_data_rwy = flight_data_rwy.drop(columns=['time_diff'])

        # Handle gaps in trajectory (possible aborted takeoff)
        while not gap_index.empty:
            first_gap_index = gap_index[0]
            if first_gap_index > 0 and first_gap_index - 1 < len(flight_data_rwy):
                timestamp_time = flight_data_rwy["timestamp"].loc[first_gap_index - 1]
            else:
                timestamp_time = None

            flight_data_rwy = flight_data_rwy.loc[first_gap_index:]

            if flight_data_rwy.empty:
                return {
                    'Event': 'Aborted Takeoff',
                    'Runway': str(rwy),
                    'Flight (Callsign)': flight.callsign,
                    'Flight ID': flight.flight_id,
                    'A/C Type': flight.typecode,
                    'ICAO 24': flight.icao24,
                    'Entry Time': first_timestamp,
                    'Not Vacated': has_no_end_data,
                    'ROT [s]': rot,
                }

            first_timestamp = pd.to_datetime(flight_data_rwy['timestamp'].min())
            last_timestamp = pd.to_datetime(flight_data_rwy['timestamp'].max())
            rot = (last_timestamp - first_timestamp).total_seconds()
            gap_index = gap_index[1:]

        if not flight_data_rwy.empty:
            entry_time = flight_data_rwy['timestamp'].iloc[0]

        if not entry_points.empty and timestamp_time is not None:
            entry_points = entry_points.loc[entry_points["timestamp"] > timestamp_time]

        # Fallback if entry points not found
        if entry_points.empty:
            points_entry = []
            for _, row in flight.data[flight.data["timestamp"] < entry_time].iterrows():
                lat = row['latitude']
                lon = row['longitude']
                point = Point(lat, lon)
                if entry_polygon.contains(point) or entry_polygon.intersects(point):
                    points_entry.append(row)
            entry_points = pd.DataFrame(points_entry)

        if not entry_points.empty:
            entry_points = entry_points[entry_points["timestamp"] < entry_time]
            if not entry_points.empty:
                entry_points["Polygon"] = entry_points.apply(lambda row: find_polygon(row["latitude"], row["longitude"], dict_ret), axis=1)
                most_common_polygon = entry_points.iloc[-required_entry_points:]["Polygon"].value_counts().idxmax()
        else:
            most_common_polygon = 'Unknown'

        return {
            "Event": "Takeoff",
            "Runway": str(rwy),
            'Flight (Callsign)': flight.callsign,
            'Flight ID': flight.flight_id,
            'A/C Type': flight.typecode,
            'ICAO 24': flight.icao24,
            'Entry Time': first_timestamp,
            'Exit Time': last_timestamp,
            'Not Vacated': has_no_end_data,
            'RET': most_common_polygon,
            'ROT [s]': rot,
        }

    return None


##### Define general Function to determine ROT of arrival Flights and further features

In [ ]:
# Define the processing function for each flight
def process_flight_arr(flight, rwy, runway_polygon, vacated_polygon, end_polygon, dict_ret, ga_flight_id_list,
                       altitude_threshold_runway, altitude_threshold_vacated, ga_altitude_threshold, time_th, time_min, required_vacated_points):

    # Initialize variables
    points_inside = []
    points_vacated = []
    points_indside_end = []
    last_valid_altitude = None
    vacated_points_count = 0
    go_around = False
    go_around_time = np.nan
    not_vacated = True
    short_rot = False

    flight = flight.sort_values(by='timestamp')

    # Iterate through flight data to collect position and altitude
    for _, row in flight.data.iterrows():
        lat = row['latitude']
        lon = row['longitude']
        alt = row['geoaltitude']

        if np.isnan(alt) and last_valid_altitude is not None:
            alt = last_valid_altitude

        point = Point(lat, lon)

        if (runway_polygon.contains(point) or runway_polygon.intersects(point)) and alt <= altitude_threshold_runway:
            points_inside.append(row)
            last_valid_altitude = alt

        if ((vacated_polygon.contains(point) or vacated_polygon.intersects(point)) and not (runway_polygon.contains(point) or runway_polygon.intersects(point)) and (points_inside)):
            points_vacated.append(row)
            if alt <= altitude_threshold_vacated:
                vacated_points_count += 1

        if end_polygon.contains(point) or end_polygon.intersects(point):
            points_indside_end.append(row)

        if vacated_points_count >= required_vacated_points:
            not_vacated = False
            break

    flight_data_rwy = pd.DataFrame(points_inside)
    flight_rwy_end_data = pd.DataFrame(points_indside_end)
    vacated_points = pd.DataFrame(points_vacated)

    if not flight_data_rwy.empty:
        first_timestamp = pd.to_datetime(flight_data_rwy['timestamp'].min())
        last_timestamp = pd.to_datetime(flight_data_rwy['timestamp'].max())
        rot = (last_timestamp - first_timestamp).total_seconds()

        # Re-check if ROT is suspiciously short
        if rot < time_min:
            short_rot = True
            points_inside = []
            points_vacated = []
            points_indside_end = []
            last_valid_altitude = None
            for _, row in flight.data.iterrows():
                lat = row['latitude']
                lon = row['longitude']
                alt = row['geoaltitude']

                if np.isnan(alt) and last_valid_altitude is not None:
                    alt = last_valid_altitude

                point = Point(lat, lon)

                if runway_polygon.contains(point) or runway_polygon.intersects(point):
                    points_inside.append(row)
                    last_valid_altitude = alt

                if ((vacated_polygon.contains(point) or vacated_polygon.intersects(point)) and not (runway_polygon.contains(point) or runway_polygon.intersects(point)) and (points_inside)):
                    points_vacated.append(row)

                if end_polygon.contains(point) or end_polygon.intersects(point):
                    points_indside_end.append(row)

            flight_data_rwy = pd.DataFrame(points_inside)
            flight_rwy_end_data = pd.DataFrame(points_indside_end)
            vacated_points = pd.DataFrame(points_vacated)

        if not flight_data_rwy.empty:
            first_timestamp = pd.to_datetime(flight_data_rwy['timestamp'].min())
            last_timestamp = pd.to_datetime(flight_data_rwy['timestamp'].max())
            rot = (last_timestamp - first_timestamp).total_seconds()

        has_end_data = not flight_rwy_end_data.empty
        points_above_ga_th = (flight_rwy_end_data['geoaltitude'] > ga_altitude_threshold).sum() if has_end_data else 0

        # Check for potential go-around conditions
        if (
            rot > time_th or (has_end_data and (flight.flight_id in ga_flight_id_list or points_above_ga_th >= 1))
        ):
            flight_data_rwy = flight_data_rwy.sort_values(by='timestamp')
            flight_data_rwy['time_diff'] = flight_data_rwy['timestamp'].diff().dt.total_seconds()
            gap_index = flight_data_rwy[flight_data_rwy['time_diff'] > time_th].index
            flight_data_rwy = flight_data_rwy.drop(columns=['time_diff'])

            if (gap_index.empty and (has_end_data and (flight.flight_id in ga_flight_id_list or points_above_ga_th >= 1))):
                go_around_time = flight_data_rwy.iloc[0]['timestamp']
                go_around = True
                return {
                    'Event': 'Go-around',
                    'Runway': str(rwy),
                    'Flight (Callsign)': flight.callsign,
                    'Flight ID': flight.flight_id,
                    'Entry Time': go_around_time,
                }

            if not gap_index.empty:
                go_around = True
                go_around_time = flight_data_rwy.iloc[0]['timestamp']
                # Prevent infinite loop
                max_iterations = 10
                iteration_count = 0
                while ((rot > time_th) and (iteration_count < max_iterations)):
                    iteration_count += 1
                    flight_data_rwy['time_diff'] = flight_data_rwy['timestamp'].diff().dt.total_seconds()
                    gap_index = flight_data_rwy[flight_data_rwy['time_diff'] > time_th].index
                    if gap_index.empty:
                        break
                    first_gap_index = gap_index[0]
                    flight_data_rwy = flight_data_rwy.loc[first_gap_index:]
                    if flight_data_rwy.empty:
                        return {
                            'Event': 'Go-around',
                            'Runway': str(rwy),
                            'Flight (Callsign)': flight.callsign,
                            'Flight ID': flight.flight_id,
                            'Entry Time': go_around_time,
                        }
                    flight_data_rwy = flight_data_rwy.drop(columns=['time_diff'])
                    first_timestamp = pd.to_datetime(flight_data_rwy['timestamp'].min())
                    last_timestamp = pd.to_datetime(flight_data_rwy['timestamp'].max())
                    rot = (last_timestamp - first_timestamp).total_seconds()
                    if not vacated_points.empty and "timestamp" in vacated_points.columns:
                        vacated_points = vacated_points[vacated_points["timestamp"] >= first_timestamp]

        entry_time = flight_data_rwy.iloc[0]['timestamp']

        if not vacated_points.empty:
            vacated_points = vacated_points[vacated_points["timestamp"] > entry_time]

            if not vacated_points.empty:
                if not_vacated or short_rot:
                    # Assign polygon based on vacated location
                    vacated_points["Polygon"] = vacated_points.apply(lambda row: find_polygon(row["latitude"], row["longitude"], dict_ret), axis=1)
                    most_common_polygon = vacated_points.iloc[:required_vacated_points]["Polygon"].value_counts().idxmax()
                else:
                    vacated_points["Polygon"] = vacated_points.apply(lambda row: find_polygon(row["latitude"], row["longitude"], dict_ret), axis=1)
                    most_common_polygon = vacated_points["Polygon"].value_counts().idxmax()
            else:
                most_common_polygon = 'Unknown'
        else:
            most_common_polygon = 'Unknown'

        # Final output for successful landing
        return {
            'Event': 'Landing',
            'Runway': str(rwy),
            'Flight (Callsign)': flight.callsign,
            'Flight ID': flight.flight_id,
            'A/C Type': flight.typecode,
            'ICAO 24': flight.icao24,
            'Entry Time': entry_time,
            'Exit Time': last_timestamp,
            'Go-around': go_around,
            'Time of Go-around': go_around_time,
            'Emergency': flight.emergency().has(),
            'Not Vacated': not_vacated,
            'Short ROT': short_rot,
            'RET': most_common_polygon,
            'ROT [s]': rot,
        }

    return None


#### Specific function for ROT determination of departure Flights

##### Runway 10

In [ ]:
def process_flight_dep_10(flight):
    return process_flight_dep(flight,
                              10, # Runway Number for Runway Entry in Table
                              runway_polygon_10, # Runway Polygon
                              runway_vacated_polygon_10, # Vacated Polygon
                              runway_end_polygon_10, # End Polygon
                              ret_dict_10, # RET Dictionary
                              3000, # Runway Polygon Threshold Altitude (max Altitude of a Point in the Runway Polygon)
                              1800, # End Polygon Threshold Altitude
                              60, # Max ROT allowed ROT without detailed examinations (Go-around, Fly over)
                              10, # Vacated Polygon Point Count Threshold (Number of points which are required for early stopping)
                              5, # Vacated Polygon Point Count Threshold (Number of points which are required for early stopping)
                              )

##### Runway 16

In [ ]:
def process_flight_dep_16(flight):
    return process_flight_dep(flight,
                              16, # Runway Number for Runway Entry in Table
                              runway_polygon_16, # Runway Polygon
                              runway_vacated_polygon_16, # Vacated Polygon
                              runway_end_polygon_16, # End Polygon
                              ret_dict_16, # RET Dictionary
                              3000, # Runway Polygon Threshold Altitude (max Altitude of a Point in the Runway Polygon)
                              1800, # Vacated Polygon Threshold Altitude
                              60, # Max ROT allowed ROT without detailed examinations (Go-around, Fly over)
                              10, # Vacated Polygon Point Count Threshold (Number of points which are required for early stopping)
                              10, # Vacated Polygon Point Count Threshold (Number of points which are required for early stopping)
                              )

##### Runway 28

In [ ]:
def process_flight_dep_28(flight):
    return process_flight_dep(flight,
                              28, # Runway Number for Runway Entry in Table
                              runway_polygon_28, # Runway Polygon
                              runway_vacated_polygon_28, # Vacated Polygon
                              runway_end_polygon_28, # End Polygon
                              ret_dict_28, # RET Dictionary
                              3000, # Runway Polygon Threshold Altitude (max Altitude of a Point in the Runway Polygon)
                              1800, # Vacated Polygon Threshold Altitude
                              60, # Max ROT allowed ROT without detailed examinations (Go-around, Fly over)
                              10, # Vacated Polygon Point Count Threshold (Number of points which are required for early stopping)
                              5, # Vacated Polygon Point Count Threshold (Number of points which are required for early stopping)
                              extra_end_polygon=extra_runway_end_polygon_28,
                              )

##### Runway 32

In [ ]:
def process_flight_dep_32(flight):
    return process_flight_dep(flight,
                              32, # Runway Number for Runway Entry in table
                              runway_polygon_32, # Runway Polygon
                              runway_vacated_polygon_32, # Vacated Polygon
                              runway_end_polygon_32, # End Polygon
                              ret_dict_32, # RET Dictionary
                              3000, # Runway Polygon Threshold Altitude (max Altitude of a Point in the Runway Polygon)
                              2000, # Vacated Polygon Threshold Altitude
                              60, # Max ROT allowed ROT without detailed examinations (Go-around, Fly over)
                              10, # Vacated Polygon Point Count Threshold (Number of points which are required for early stopping)
                              5, # Vacated Polygon Point Count Threshold (Number of points which are required for early stopping)
                              )

##### Runway 34

In [ ]:
def process_flight_dep_34(flight):
    return process_flight_dep(flight,
                              34, # Runway Number for Runway Entry in table
                              runway_polygon_dep_34, # Runway Polygon
                              runway_vacated_polygon_34, # Vacated Polygon
                              runway_end_polygon_34, # End Polygon
                              ret_dict_34, # RET Dictionary
                              3000, # Runway Polygon Threshold Altitude (max Altitude of a Point in the Runway Polygon)
                              2000, # Vacated Polygon Threshold Altitude
                              60, # Max ROT allowed ROT without detailed examinations (Go-around, Fly over)
                              10, # Vacated Polygon Point Count Threshold (Number of points which are required for early stopping)
                              5, # Vacated Polygon Point Count Threshold (Number of points which are required for early stopping)
                              )

#### Specific function for ROT determination of Arrival Flights

##### Runway 14

In [ ]:
def process_flight_arr_14(flight):
    return process_flight_arr(flight,
                              14, # Runway Number for Runway Entry in table
                              runway_polygon_14, # Runway Polygon
                              runway_vacated_polygon_14, # Vacated Polygon
                              runway_end_polygon_14, # End Polygon
                              ret_dict_14, # RET Dictionary
                              go_around_ids, # Go-around IDS
                              4000, # Runway Polygon Threshold Altitude (max Altitude of a Point in the Runway Polygon)
                              2000, # Vacated Polygon Threshold Altitude
                              1800, # Minimum Altitude to be considered as part of the Go-around Polygon
                              150, # Max ROT allowed ROT without detailed examinations (Go-around, Fly over)
                              40, # Minimum ROT without further investment
                              10 # Vacated Polygon Point Count Threshold (Number of points which are required for early stopping)
                              )

##### Runway 16

In [ ]:
def process_flight_arr_16(flight):
    return process_flight_arr(flight,
                              16, # Runway Number for Runway Entry in table
                              runway_polygon_16, # Runway Polygon
                              runway_vacated_polygon_16, # Vacated Polygon
                              runway_end_polygon_16, # End Polygon
                              ret_dict_16, # RET Dictionary
                              go_around_ids, # Go-around IDS
                              4000, # Runway Polygon Threshold Altitude (max Altitude of a Point in the Runway Polygon)
                              2000, # Vacated Polygon Threshold Altitude
                              1800, # Minimum Altitude to be considered as part of the Go-around Polygon
                              150, # Max ROT allowed ROT without detailed examinations (Go-around, Fly over)
                              40, # Minimum ROT without further investment
                              10 # Vacated Polygon Point Count Threshold (Number of points which are required for early stopping)
                              )

##### Runway 28

In [ ]:
def process_flight_arr_28(flight):
    return process_flight_arr(flight,
                              28, # Runway Number for Runway Entry in table
                              runway_polygon_28, # Runway Polygon
                              runway_vacated_polygon_28, # Vacated Polygon
                              runway_end_polygon_28, # End Polygon
                              ret_dict_28, # RET Dictionary
                              go_around_ids, # Go-around IDS
                              4000, # Runway Polygon Threshold Altitude (max Altitude of a Point in the Runway Polygon)
                              2000, # Vacated Polygon Threshold Altitude
                              1800, # Minimum Altitude to be considered as part of the Go-around Polygon
                              150, # Max ROT allowed ROT without detailed examinations (Go-around, Fly over)
                              30, # Minimum ROT without further investment
                              10 # Vacated Polygon Point Count Threshold (Number of points which are required for early stopping)
                              )

##### Runway 34

In [ ]:
def process_flight_arr_34(flight):
    return process_flight_arr(flight,
                              34, # Runway Number for Runway Entry in table
                              runway_polygon_34, # Runway Polygon
                              runway_vacated_polygon_34, # Vacated Polygon
                              runway_end_polygon_34, # End Polygon
                              ret_dict_34, # RET Dictionary
                              go_around_ids, # Go-around IDS
                              4000, # Runway Polygon Threshold Altitude (max Altitude of a Point in the Runway Polygon)
                              2000, # Vacated Polygon Threshold Altitude
                              1800, # Minimum Altitude to be considered as part of the Go-around Polygon
                              150, # Max ROT allowed ROT without detailed examinations (Go-around, Fly over)
                              30, # Minimum ROT without further investment
                              10 # Vacated Polygon Point Count Threshold (Number of points which are required for early stopping)
                              )

##### Evaluate Function on Trajs and add data to table

##### departures

##### Runway 10

In [ ]:
# Lazy iteration over flights using iterate_lazy
table_dep_10 = trajs_dep_10.iterate_lazy().pipe(process_flight_dep_10).eval(max_workers=40, desc='Processing Flights')
del trajs_dep_10

##### Runway 16

In [ ]:
# Lazy iteration over flights using iterate_lazy
table_dep_16 = trajs_dep_16.iterate_lazy().pipe(process_flight_dep_16).eval(max_workers=40, desc='Processing Flights')
del trajs_dep_16

##### Runway 28

In [ ]:
# Lazy iteration over flights using iterate_lazy
table_dep_28 = trajs_dep_28.iterate_lazy().pipe(process_flight_dep_28).eval(max_workers=40, desc='Processing Flights')
del trajs_dep_28

##### Runway 32

In [ ]:
# Lazy iteration over flights using iterate_lazy
table_dep_32 = trajs_dep_32.iterate_lazy().pipe(process_flight_dep_32).eval(max_workers=40, desc='Processing Flights')
del trajs_dep_32

##### Runway 34

In [ ]:
# Lazy iteration over flights using iterate_lazy
table_dep_34 = trajs_dep_34.iterate_lazy().pipe(process_flight_dep_34).eval(max_workers=40, desc='Processing Flights')
del trajs_dep_34

##### Arrivals

##### Runway 14

In [ ]:
# Lazy iteration over flights using iterate_lazy
table_arr_14 = trajs_arr_14.iterate_lazy().pipe(process_flight_arr_14).eval(max_workers=40, desc='Processing Flights')
del trajs_arr_14

##### Runway 16

In [ ]:
# Lazy iteration over flights using iterate_lazy
table_arr_16 = trajs_arr_16.iterate_lazy().pipe(process_flight_arr_16).eval(max_workers=40, desc='Processing Flights')
del trajs_arr_16

##### Runway 28

In [ ]:
# Lazy iteration over flights using iterate_lazy
table_arr_28 = trajs_arr_28.iterate_lazy().pipe(process_flight_arr_28).eval(max_workers=40, desc='Processing Flights')
del trajs_arr_28

##### Runway 34

In [ ]:
# Lazy iteration over flights using iterate_lazy
table_arr_34 = trajs_arr_34.iterate_lazy().pipe(process_flight_arr_34).eval(max_workers=40, desc='Processing Flights')
del trajs_arr_34

In [ ]:
table = pd.concat([table_dep_10, table_dep_16, table_dep_28, table_dep_32, table_dep_34, table_arr_14, table_arr_16, table_arr_28, table_arr_34], ignore_index=True)

In [ ]:
del table_dep_10
del table_dep_16
del table_dep_28
del table_dep_32
del table_dep_34
del table_arr_14
del table_arr_16
del table_arr_28
del table_arr_34

##### Creating an own Entry for Go-Arounds

In [ ]:
# Duplicate rows where 'Time of Go-around' has a value
go_around_rows = table.dropna(subset=['Time of Go-around']).copy()

# Keep only the required columns and rename 'Time of Go-around' to 'Entry Time'
go_around_rows = go_around_rows[['Runway', 'Flight (Callsign)', 'Flight ID', 'Time of Go-around']].copy()
go_around_rows.rename(columns={'Time of Go-around': 'Entry Time'}, inplace=True)

# Set the 'Event' column to 'Go-around'
go_around_rows['Event'] = 'Go-around'

# Concatenate the original and new rows, then sort by 'Flight ID' and 'Entry Time'
table = pd.concat([table, go_around_rows], ignore_index=True).sort_values(by=['Flight ID', 'Entry Time'])

##### Sort table in regards of Time at TH

In [ ]:
# Sort values
table = table.sort_values(by='Entry Time')

# Reset index
table.reset_index(drop=True, inplace=True)

table


### Import and Add Data from FAA Aircraft Characteristics Database

##### In case of missing data in opensky, an manually built database is used to add ICAO Aircraft Type

In [ ]:
# Load the list from the CSV file into a DataFrame
DF_MISSING_ICAO24 = pd.read_csv('missing_icao24_data.csv')

##### Add and check A/C Type for missing  or outdated entries by using a manual database

In [ ]:
# Create a dictionary from the second table (DF_MISSING_ICAO24) for quick access
icao24_dict = DF_MISSING_ICAO24.set_index('icao24').to_dict(orient='index')

# Update 'A/C Type' for missing or outdated values
for icao24, info in icao24_dict.items():
    # Mask for rows with matching icao24 and either missing or outdated 'A/C Type'
    mask_ac_type = (table['ICAO 24'] == icao24) & ((table['A/C Type'].isna()) | (table['A/C Type'] == '') | (table['A/C Type'] != info['typecode']))
    table.loc[mask_ac_type, 'A/C Type'] = info['typecode']

# Display the result for 'A/C Type' updates
table

##### Load the FAA Characterisitcs Database Excel file into a DataFrame

In [ ]:
aircraft_characteristics = pd.read_excel('www.faa.gov.xlsx')

##### Add Maximum Allowable Landing Weight (MALW) anc ICAO Weight Turbulence Category (WTC) to table

In [ ]:
table = table.merge(aircraft_characteristics[['ICAO_Code', 'MALW_lb', 'ICAO_WTC']],
                                  left_on='A/C Type',    # Column in table
                                  right_on='ICAO_Code',  # Column in aircraft_characteristics
                                  how='left').drop(columns='ICAO_Code')  # Optionally drop 'ICAO_Code' after merge


In [ ]:
table

##### Convert to metric units

In [ ]:
# Convert non-NaN values from pounds to kilograms and round, keeping NaNs as they are
table['MALW [kg]'] = table['MALW_lb'].apply(lambda x: round(x * 0.453592) if pd.notna(x) else x)

### Engineer further Data from table

##### Add ICAO Aircraft Type table

In [ ]:
table

In [ ]:
# Perform the merge
table = table.merge(aircraft.data[['icao24', 'icaoaircrafttype']],
                                  left_on='ICAO 24',
                                  right_on='icao24',  # Column in aircraft.data
                                  how='left')

##### In case of missing data in opensky, an manually built database is used to add ICAO Aircraft Type

In [ ]:
# Update 'icaoaircrafttype' for missing or outdated values
for icao24, info in icao24_dict.items():
    # Mask for rows with matching icao24 and either missing or outdated 'icaoaircrafttype'
    mask_icaoaircrafttype = (table['ICAO 24'] == icao24) & ((table['icaoaircrafttype'].isna()) | (table['icaoaircrafttype'] == '') | (table['icaoaircrafttype'] != info['icaoaircrafttype']))
    table.loc[mask_icaoaircrafttype, 'icaoaircrafttype'] = info['icaoaircrafttype']

# Display the result for 'icaoaircrafttype' updates
table

##### Extract Number of Engines and Propulsion Type from ICAO Aircraft Code

In [ ]:
# Function to extract number of engines and propulsion type dynamically based on length
def get_engine_info(icao_type):
    if isinstance(icao_type, str) and len(icao_type) == 3:  # Check if the length is exactly 3
        num_engines = int(icao_type[1])  # Extract number of engines (2nd character) and convert to integer
        propulsion_code = icao_type[2]   # Extract propulsion type (3rd character)

        # Check for propulsion type based on the 3rd character
        if propulsion_code == 'J':
            propulsion_type = 'Jet'
        elif propulsion_code == 'T':
            propulsion_type = 'Turboprop'
        elif propulsion_code == 'P':
            propulsion_type = 'Piston'
        else:
            propulsion_type = np.nan  # Handle unknown propulsion types

        return propulsion_type, num_engines
    else:
        return np.nan, np.nan  # Handle cases where length is not exactly 3

# Apply the function to the 'icaoaircrafttype' column
table['Propulsion Type'], table['Number of Engines'] = zip(
    *table['icaoaircrafttype'].apply(get_engine_info)
)

# Ensure NaN instead of <NA> in the 'Number of Engines' column
table['Number of Engines'] = table['Number of Engines'].replace({pd.NA: np.nan})

# Convert "Number of Engines" to integer type
table['Number of Engines'] = table['Number of Engines'].astype('float64')  # Use float to handle NaN

# Display the result
table


##### Derive ICAO Code from callsign (Airline)

In [ ]:
# Step 1: Extract the first three characters of the 'Flight (Callsign)' column to get the ICAO Code
table['ICAO Code'] = table['Flight (Callsign)'].str[:3]

# Step 2: Define a threshold for the minimum number of occurrences required for an ICAO Code to be considered valid
threshold = 0  # Set your desired threshold value here

# Step 3: Count occurrences of each ICAO Code in the table
icao_counts = table['ICAO Code'].value_counts()

# Step 4: Assign "Unknown" to ICAO Codes that do not meet the threshold
table['ICAO Code'] = table['ICAO Code'].apply(lambda x: x if icao_counts.get(x, 0) >= threshold else 'Unknown')


##### Determine Entry Time Difference between succesive landings

In [ ]:
# Convert 'Entry Time' to datetime format and force conversion with 'coerce' to mark non-date values as NaT
table['Entry Time'] = pd.to_datetime(table['Entry Time'], errors='coerce')

# Check if all values were correctly converted
if table['Entry Time'].isnull().any():
    print("Warning: Some values in 'Entry Time' could not be converted and were marked as NaT.")

# Filter only landings
landings = table[table['Event'] == 'Landing']

# Calculate the time difference to the previous landing on the same runway
table['Time from Previous Landing [s]'] = (
    table['Entry Time'] - landings.groupby('Runway')['Entry Time'].shift(1)
).dt.total_seconds()

# Fill NaN values (e.g., first landing on each runway has no previous landing)
table['Time from Previous Landing [s]'] = table['Time from Previous Landing [s]'].fillna(0)


##### Time from Previous Event

In [ ]:
# Calculate the time difference to the previous event on the same runway
table['Time from Previous Event [s]'] = (
    table['Entry Time'] - table.groupby('Runway')['Entry Time'].shift(1)
).dt.total_seconds()

# Fill NaN values (e.g., first event on each runway has no previous event)
table['Time from Previous Event [s]'] = table['Time from Previous Event [s]'].fillna(0)


##### Time to next Event

In [ ]:
# Calculate time to the next event on the same runway
table['Time to Next Event [s]'] = (
    table.groupby('Runway')['Entry Time']
    .shift(-1) - table['Entry Time']
)

# Convert to seconds
table['Time to Next Event [s]'] = table['Time to Next Event [s]'].dt.total_seconds()

##### Determine Gate

In [ ]:
table_reduced = table.loc[(table['Event'] == 'Takeoff') | (table['Event'] == 'Landing')]
table_reduced_flight_ids = table_reduced['Flight ID'].unique().tolist()
trajs_gate = trajs[table_reduced_flight_ids]

In [ ]:
def find_gate(flight):
    """Finds the most frequent gate region (North, East, South, West) based on first or last 5 recorded positions."""
    num_points = 5  # Number of points to check

    # Retrieve the event type (Takeoff or Landing) for the given flight
    event_series = table_reduced['Event'].loc[table_reduced['Flight ID'] == flight.flight_id]

    if event_series.empty:
        return None  # No event data found

    event = event_series.iloc[0]

    if event == "Takeoff":
        points = flight.data.iloc[:num_points][["longitude", "latitude"]].values
    elif event == "Landing":
        points = flight.data.iloc[-num_points:][["longitude", "latitude"]].values
    else:
        return None  # Unknown event

    gate_counter = Counter()

    for lon, lat in points:
        point = Point(lat, lon)

        for gate_name, polygon in gate_polygons.items():
            if polygon.contains(point):
                gate_counter[gate_name] += 1
                break  # Found a matching polygon, stop checking

    if not gate_counter:
        return {
            "Flight ID": flight.flight_id,
            "Gate Region": "Unknown"
        }

    most_common_gate = gate_counter.most_common(1)[0][0]

    return {
        "Flight ID": flight.flight_id,
        "Gate Region": most_common_gate
    }


In [ ]:
gate_table = trajs_gate.iterate_lazy().pipe(find_gate).eval(max_workers=40, desc="Processing")
table = table.merge(gate_table, on="Flight ID", how="left")

##### Rename the columns

In [ ]:
table = table.rename(columns={
    'icao24': 'ICAO 24',
    'icaoaircrafttype': 'ICAO Aircraft Type',
    'ICAO_WTC': 'ICAO Weight Turbulence Category'}
)

#### Add missing aircraft characteristics data

In [ ]:
missing_aircraft_characteristics = pd.read_csv('missing_malw.csv')

In [ ]:
# Rename column to match the naming in the main table
df_missing = missing_aircraft_characteristics.rename(columns={'IWTC': 'ICAO Weight Turbulence Category'})

# Keep only the relevant columns
df_missing = df_missing[['A/C Type', 'MALW [kg]', 'ICAO Weight Turbulence Category']]

# Merge to supplement missing values
merged = table.merge(df_missing, on='A/C Type', how='left', suffixes=('', '_missing'))

# Fill missing values in the main table with values from the merged dataframe
table['MALW [kg]'] = table['MALW [kg]'].fillna(merged['MALW [kg]_missing'])
table['ICAO Weight Turbulence Category'] = table['ICAO Weight Turbulence Category'].fillna(
    merged['ICAO Weight Turbulence Category_missing']
)


### Rearrange the columns as definded at the beginning

In [ ]:
table = table[columns]
table

##### Get required Flight Data

In [ ]:
table_required = table.loc[(table['Event'] == 'Landing') & (table['Runway'] == '14')]
flight_ids_required = table_required['Flight ID'].unique()

trajs_required_complete = trajs[flight_ids_required].resample('1s').eval(max_workers=40, desc='Processing')

##### Save complete required dataset as parquet file

In [ ]:
trajs_required_complete.to_parquet('trajs_required_complete.parquet')

##### Get Airborne Data before Entry Time

In [ ]:
def required_airborne_part(flight):
    entry_time = table_required['Entry Time'].loc[table['Flight ID'] == flight.flight_id]
    entry_time = pd.to_datetime(entry_time)

    if not entry_time.empty:
        entry_time = entry_time.iloc[0]
        flight.data = flight.data.loc[flight.data['timestamp'] < entry_time]
    
    return flight


In [ ]:
trajs_required_airborne = trajs_required_complete.iterate_lazy().pipe(required_airborne_part).eval(max_workers=40,desc="Processing")

##### Save airborne required dataset as parquet file

In [ ]:
trajs_required_airborne.to_parquet('trajs_required_airborne.parquet')

##### Reduce Dataset only up to prediction point

In [ ]:
threshold_14 = airports['LSZH'].runways.list[2]
trajs_required = trajs_required_airborne.distance(threshold_14).eval(max_workers=1, desc='Processing')

In [ ]:
trajs_required = trajs_required.query("distance >= 4")

##### Save required dataset up to prediction point as parquet file

In [ ]:
trajs_required.to_parquet('trajs_required_prediction.parquet')

##### Get only Data Snippet directly in front of prediction point

In [ ]:
trajs_required = trajs_required.last('10s').eval(max_workers=40, desc='Processing')

##### Save final trajectory snippet dataset

In [ ]:
trajs_required.to_parquet('Trajectory_Snippet.parquet')

In [ ]:
final_table = table_required[['Flight ID', 'Entry Time', 'Exit Time', 'ICAO Code', 'A/C Type', 'ICAO Aircraft Type', 'Propulsion Type', 'Number of Engines', 'MALW [kg]', 'ICAO Weight Turbulence Category', 'Gate Region', 'RET', 'ROT [s]']]

#### Time of Prediction

In [ ]:
def time_of_prediction(flight):
    top = flight.stop
    return{
        "Flight ID": flight.flight_id,
        "Time of Prediction": top
    }

In [ ]:
prediction_time_table = trajs_required.iterate_lazy().pipe(time_of_prediction).eval(max_workers=40, desc="Processing Flights")

##### Add Time of Prediction to Final Table

In [ ]:
final_table = final_table.merge(prediction_time_table, on="Flight ID", how="outer")

##### Sort Final Final Table by Time of Prediction

In [ ]:
final_table = final_table.sort_values('Time of Prediction')

##### Remove Flights without Time of Prediction

In [ ]:
final_table = final_table.loc[~final_table['Time of Prediction'].isna()]
final_table = final_table.reset_index(drop=True)

### Add Weather Data to Final Final Table

##### Define Function to find the closest METAR

In [ ]:
# Function to find the closest METAR to a timestamp of a flight in the past or at the same time
def find_closest_past_metar(timestamp, meteo_times):
    # Calculate time difference
    time_diff = meteo_times - timestamp
    # Filter out future METAR times (i.e., positive time differences)
    past_times = time_diff[time_diff <= pd.Timedelta(0)]
    
    # If there are no past times available, return None or handle accordingly
    if past_times.empty:
        return None

    # Calculate absolute differences and find the closest
    closest_index = past_times.abs().idxmin()  # Find closest index from past times
    return closest_index

##### Import of all METARS within Time periode

In [ ]:
meteo_data = []
for t1, t2 in zip(timestamps[:-1], timestamps[1:]):
        print(t1, t2)
        tmp = metar.METAR('LSZH').get(
             start=str(t1),
             stop=str(t2),
        )
        if tmp is not None:
            meteo_data.append(tmp)       
 
meteo = pd.concat(meteo_data)

# Sort the DataFrame by time and reset the index
meteo = meteo.sort_values(by="time").reset_index(drop=True)

##### Add weather data for each flight to the final Final Table based on the closest METAR

In [ ]:
# Loop through each row in the DataFrame
for flight in range(len(final_table)):
    # Convert the time column to datetime objects
    time = pd.to_datetime(final_table['Time of Prediction'].iloc[flight])
    
    # Find the index of the closest METAR report
    idx = find_closest_past_metar(time, meteo['time'])
    if not idx == None:
    
        # Get the corresponding values from the METAR DataFrame
        wind_spd = meteo.wind_speed[idx]
        wind_dir = meteo.wind_dir[idx]
        visibility = meteo.vis[idx]
        temperature = meteo.temp[idx]
        
        # Insert the values into the corresponding columns in 'final_table'
        final_table.at[flight, 'Wind speed [kt]'] = wind_spd
        final_table.at[flight, 'Wind direction [°]'] = wind_dir
        final_table.at[flight, 'Visibility Category'] = visibility
        final_table.at[flight, 'Temperature [°C]'] = temperature
        final_table.at[flight, 'Meteo idx'] = idx


##### Convert Temperature, Wind speed and Wind direction Data in Final Table to numerical values

In [ ]:
# Convert all values in the relevant columns to strings
final_table['Temperature [°C]'] = final_table['Temperature [°C]'].astype(str)
final_table['Wind speed [kt]'] = final_table['Wind speed [kt]'].astype(str)
final_table['Wind direction [°]'] = final_table['Wind direction [°]'].astype(str)

# Clean the columns (e.g., remove leading/trailing spaces)
final_table['Temperature [°C]'] = final_table['Temperature [°C]'].str.strip()
final_table['Wind speed [kt]'] = final_table['Wind speed [kt]'].str.strip()
final_table['Wind direction [°]'] = final_table['Wind direction [°]'].str.strip()

# Replace empty strings or 'None' with NaN in the 'Wind direction [°]' column
final_table['Wind direction [°]'] = final_table['Wind direction [°]'].replace(['None', ''], pd.NA)

# Extract the numeric values (keeping negative numbers!)
final_table['Temperature [°C]'] = final_table['Temperature [°C]'].str.extract(r'(-?\d+\.?\d*)')
final_table['Wind speed [kt]'] = final_table['Wind speed [kt]'].str.extract(r'(\d+\.?\d*)')
final_table['Wind direction [°]'] = final_table['Wind direction [°]'].str.extract(r'(\d+\.?\d*)')

# Convert the extracted values to floats
final_table['Temperature [°C]'] = pd.to_numeric(final_table['Temperature [°C]'], errors='coerce').astype(float)
final_table['Wind speed [kt]'] = pd.to_numeric(final_table['Wind speed [kt]'], errors='coerce').astype(float)
final_table['Wind direction [°]'] = pd.to_numeric(final_table['Wind direction [°]'], errors='coerce').astype(float)


##### Define Visibility categories

In [ ]:
# Function to categorize visibility into categories
def categorize_visibility(visibility):
    if visibility <= 1000:
        return 0  # Very poor
    elif visibility <= 2000:
        return 1  # Poor
    elif visibility <= 3000:
        return 2
    elif visibility <= 4000:
        return 3
    elif visibility <= 5000:
        return 4
    elif visibility <= 6000:
        return 5
    elif visibility <= 7000:
        return 6
    elif visibility <= 8000:
        return 7
    elif visibility <= 9000:
        return 8
    elif visibility <= 10000:
        return 9  # Good
    else:
        return 10  # Very good

##### Transfer Visibility Data in final Final Table to visibility category

In [ ]:
# Convert all values in the 'Visibility Category' column to strings
final_table['Visibility Category'] = final_table['Visibility Category'].astype(str)

# Clean the 'Visibility Category' column (e.g., remove leading/trailing spaces)
final_table['Visibility Category'] = final_table['Visibility Category'].str.strip()

# Replace empty strings or 'None' with NaN for better compatibility with ML
final_table['Visibility Category'] = final_table['Visibility Category'].replace(['None', ''], pd.NA)

# Conditional processing: If 'greater than' is present, set the value to '12000',
# otherwise extract the number as an integer or float
final_table['Visibility Category'] = final_table['Visibility Category'].apply(
    lambda x: 12000 if isinstance(x, str) and 'greater than' in x.lower() else x
)

# For all non-'greater than' values, extract the number and keep it as a float (NaN for missing)
final_table['Visibility Category'] = final_table['Visibility Category'].apply(
    lambda x: pd.to_numeric(x.split()[0], errors='coerce') if isinstance(x, str) else x
)

# Ensure the final column is numeric with NaN for missing values
final_table['Visibility Category'] = pd.to_numeric(final_table['Visibility Category'], errors='coerce')


# Apply the visibility categorization directly to the 'Visibility Category' column
final_table['Visibility Category'] = final_table['Visibility Category'].apply(categorize_visibility)


##### Import Precipitation (Open-Meteo)

In [ ]:
# Coordinates for Zurich Airport RWY 14
latitude = 47.473626
longitude = 8.547640

# Convert start and stop times to datetime format
start_datetime = pd.to_datetime(start)
stop_datetime = pd.to_datetime(stop)

# Extract the date in "YYYY-MM-DD" format
start_date = start_datetime.strftime("%Y-%m-%d")
end_date = stop_datetime.strftime("%Y-%m-%d")
hourly = "precipitation"

# Fetch weather data from Open-Meteo
url = f"https://archive-api.open-meteo.com/v1/era5?latitude={latitude}&longitude={longitude}&start_date={start_date}&end_date={end_date}&hourly={hourly}"
response = requests.get(url)
data = response.json()

# Create a Pandas DataFrame with timestamps and precipitation data
meteo_df = pd.DataFrame({
    'Timestamp': pd.to_datetime(data['hourly']['time']),
    'Precipitation [mm]': data['hourly']['precipitation']
})

# Optional: Ensure the timestamps are localized to UTC if needed
meteo_df['Timestamp'] = meteo_df['Timestamp'].dt.tz_localize('UTC')


##### Define function to find clostes Precipitation Data

In [ ]:
# Function to determine the nearest precipitation value to the flight timestamp in the past or at the same time
def find_closest_past_precipitation(flight_time, meteo_df):
    # Filter out future precipitation times (i.e., only keep timestamps in the past or at the same time)
    past_times_df = meteo_df[meteo_df['Timestamp'] <= flight_time]

    # If there are no past times available, return None
    if past_times_df.empty:
        return None

    # Calculate the time differences
    time_diff = (past_times_df['Timestamp'] - flight_time).abs()

    # Find the index of the minimum time difference (i.e., closest past time)
    closest_index = time_diff.idxmin()
    
    return closest_index

##### Add closest Precipitation Data to Final Table

In [ ]:
# Loop through each flight time in 'final_table' to find the closest precipitation value
for flight in range(len(final_table)):
    # Temporary conversion of 'Time of Prediction' to datetime without changing the original final_table format
    flight_time = pd.to_datetime(final_table['Time of Prediction'].iloc[flight], errors='coerce')
    
    # Check if the conversion was successful
    if pd.notna(flight_time):  # Valid datetime object
        # Find the index of the closest precipitation data in the past or at the same time
        idx = find_closest_past_precipitation(flight_time, meteo_df)

        if idx is not None:
            # Get the corresponding precipitation value and timestamp
            precipitation_value = meteo_df.loc[idx, 'Precipitation [mm]']
            corresponding_timestamp = meteo_df.loc[idx, 'Timestamp']

            # Insert the precipitation value and timestamp into the corresponding columns in 'final_table'
            final_table.at[flight, 'Precipitation [mm]'] = precipitation_value
            final_table.at[flight, 'Precipitation idx'] = idx
            final_table.at[flight, 'Precipitation Timestamp'] = corresponding_timestamp
        else:
            # No valid past time found, set value to None
            final_table.at[flight, 'Precipitation [mm]'] = None
            final_table.at[flight, 'Precipitation idx'] = None
            final_table.at[flight, 'Precipitation Timestamp'] = None

    else:
        # Handle cases where datetime conversion fails
        final_table.at[flight, 'Precipitation [mm]'] = None
        final_table.at[flight, 'Precipitation idx'] = None
        final_table.at[flight, 'Precipitation Timestamp'] = None


##### Cyclic encoding of Time of Prediction

In [ ]:
# Sample structure based on your 'Time of Prediction' column
final_table['Hour'] = final_table['Time of Prediction'].dt.hour
final_table['Minute'] = final_table['Time of Prediction'].dt.minute
final_table['Day of week'] = final_table['Time of Prediction'].dt.weekday
final_table['Month'] = final_table['Time of Prediction'].dt.month
final_table['Year'] = final_table['Time of Prediction'].dt.year % 100  # Using modulo 100 for 'year' cyclic encoding

# Function to apply encoding to one column
def cyclic_enc(df, feature, max_val):
    df[feature + " sin"] = (
        0.5 * np.sin(2 * np.pi * df[feature] / max_val) + 0.5
    )
    df[feature + " cos"] = (
        0.5 * np.cos(2 * np.pi * df[feature] / max_val) + 0.5
    )
    return df

def encode_df(df):
    # Cyclic encoding of hour, minute, weekday, month, and year
    df = cyclic_enc(df, 'Hour', 24)
    df = cyclic_enc(df, 'Minute', 60)
    df = cyclic_enc(df, 'Day of week', 7)
    df = cyclic_enc(df, 'Month', 12)
    df = cyclic_enc(df, 'Year', 100)
    
    # Drop original columns after encoding
    df = df.drop(columns=['Hour', 'Minute', 'Day of week', 'Month', 'Year'])
    return df

# Apply encoding to the dataset
final_table = encode_df(final_table)


##### Remove Flights without Time of Prediction

In [ ]:
final_table = final_table.loc[~final_table['Time of Prediction'].isna()]

##### Determine Day or Night Status Based on Sun's Position at 'Time of Prediction'

In [ ]:
# Define location information (Zurich Airport)
location = LocationInfo("Zurich Airport", "Switzerland", "Europe/Zurich", latitude, longitude)

# Function to determine if it is day or night based on the timestamp
def is_daytime(timestamp):
    # Calculate sunrise and sunset times for the date of the timestamp
    s = sun(location.observer, date=timestamp.date())
    sunrise = s['sunrise']
    sunset = s['sunset']
    
    # Check if the timestamp is between sunrise and sunset
    return sunrise <= timestamp <= sunset

# Apply the function to the 'Time of Prediction' column and create a new 'Day/Night' column
final_table['Day/Night'] = final_table['Time of Prediction'].apply(lambda x: 'Day' if is_daytime(x) else 'Night')


##### Add additional Wind data and cyclic encode wind data

In [ ]:
# 1. `No Wind`: Wind speed is 0
final_table['No Wind'] = final_table['Wind speed [kt]'] == 0

# 2. `Wind Variable`: Wind direction is NaN, but wind speed > 0
final_table['Wind Variable'] = final_table.apply(
    lambda row: pd.isna(row['Wind direction [°]']) and row['Wind speed [kt]'] > 0,
    axis=1
)

# 3. Transform wind direction into sine/cosine
# Process only valid wind directions
final_table['Wind direction sin_raw'] = np.sin(np.deg2rad(final_table['Wind direction [°]']))
final_table['Wind direction cos_raw'] = np.cos(np.deg2rad(final_table['Wind direction [°]']))

# 4. Scale sine/cosine values to the range [0, 1]
final_table['Wind direction sin'] = (final_table['Wind direction sin_raw'] + 1) / 2
final_table['Wind direction cos'] = (final_table['Wind direction cos_raw'] + 1) / 2

# Set sine/cosine values to 0 for "No Wind" and "Wind Variable"
final_table.loc[final_table['No Wind'] | final_table['Wind Variable'], ['Wind direction sin', 'Wind direction cos']] = 0

# Remove unnecessary intermediate columns
final_table.drop(columns=['Wind direction sin_raw', 'Wind direction cos_raw'], inplace=True)

final_table

##### Determine Head- and Crosswind Components based on Wind Data

In [ ]:
runway_direction = airports['LSZH'].runways.data.bearing.loc[2]

# Convert runway direction to radians
runway_rad = np.radians(runway_direction)

# Calculate and round Headwind and Crosswind to two decimal places
final_table['Headwind [kt]'] = final_table.apply(
    lambda row: round(row['Wind speed [kt]'] * np.cos(np.radians(row['Wind direction [°]']) - runway_rad), 2) 
    if pd.notna(row['Wind speed [kt]']) and pd.notna(row['Wind direction [°]']) else np.nan, axis=1
)

final_table['Crosswind [kt]'] = final_table.apply(
    lambda row: round(row['Wind speed [kt]'] * np.sin(np.radians(row['Wind direction [°]']) - runway_rad), 2) 
    if pd.notna(row['Wind speed [kt]']) and pd.notna(row['Wind direction [°]']) else np.nan, axis=1
)
# Display the updated DataFrame to confirm the new columns
final_table

##### Determine Airport Runway Movements in the last 15 Minutes prior to the Prediction Time

In [ ]:
def runway_movements(flight):
    time_interval = 15 # minutes
    time_interval = pd.Timedelta(minutes=time_interval)
    prediction_time = flight.stop

    if not prediction_time == None:
        lower_time_bound = prediction_time-time_interval
        event_table_interval = table[(table['Entry Time'] >= lower_time_bound) & (table['Entry Time'] < prediction_time)]

        total_events_table_rwy_10 = event_table_interval[event_table_interval['Runway'] == '10']
        total_events_table_rwy_14 = event_table_interval[event_table_interval['Runway'] == '14']
        total_events_table_rwy_16 = event_table_interval[event_table_interval['Runway'] == '16']
        total_events_table_rwy_28 = event_table_interval[event_table_interval['Runway'] == '28']
        total_events_table_rwy_32 = event_table_interval[event_table_interval['Runway'] == '32']
        total_events_table_rwy_34 = event_table_interval[event_table_interval['Runway'] == '34']


        total_arrivals_table = event_table_interval[(event_table_interval['Event'] == 'Landing') | (event_table_interval['Event'] == 'Go-around')]

        total_arrivals_rwy_16 = total_arrivals_table[total_arrivals_table['Runway'] == '16']
        total_arrivals_rwy_28 = total_arrivals_table[total_arrivals_table['Runway'] == '28']
        total_arrivals_rwy_34 = total_arrivals_table[total_arrivals_table['Runway'] == '34']

        total_departures_table = event_table_interval[event_table_interval['Event'] == 'Takeoff']

        total_departures_rwy_16 = total_departures_table[total_departures_table['Runway'] == '16']
        total_departures_rwy_28 = total_departures_table[total_departures_table['Runway'] == '28']
        total_departures_rwy_34 = total_departures_table[total_departures_table['Runway'] == '34']

        total_rwy_10_events = len(total_events_table_rwy_10)
        total_rwy_14_events = len(total_events_table_rwy_14)

        departures_rwy_16 = len(total_departures_rwy_16)
        arrivals_rwy_16 = len(total_arrivals_rwy_16)
        total_rwy_16_events = len(total_events_table_rwy_16)

        departures_rwy_28 = len(total_departures_rwy_28)
        arrivals_rwy_28 = len(total_arrivals_rwy_28)
        total_rwy_28_events = len(total_events_table_rwy_28)

        total_rwy_32_events = len(total_events_table_rwy_32)

        departures_rwy_34 = len(total_departures_rwy_34)
        arrivals_rwy_34 = len(total_arrivals_rwy_34)
        total_rwy_34_events = len(total_events_table_rwy_34)

        departures_airport_events = len(total_departures_table)
        arrivals_airport_events = len(total_arrivals_table)
        total_airport_events = len(event_table_interval)

        return {
            "Flight ID": flight.flight_id,
            "Traffic Intensity RWY 10": total_rwy_10_events,
            "Traffic Intensity RWY 14": total_rwy_14_events,
            "Departure Traffic Intensity RWY 16": departures_rwy_16,
            "Arrival Traffic Intensity RWY 16": arrivals_rwy_16,
            "Traffic Intensity RWY 16": total_rwy_16_events,
            "Departure Traffic Intensity RWY 28": departures_rwy_28,
            "Arrival Traffic Intensity RWY 28": arrivals_rwy_28,
            "Traffic Intensity RWY 28": total_rwy_28_events,
            "Traffic Intensity RWY 32": total_rwy_32_events,
            "Departure Traffic Intensity RWY 34": departures_rwy_34,
            "Arrival Traffic Intensity RWY 34": arrivals_rwy_34,
            "Traffic Intensity RWY 34": total_rwy_34_events,
            "Airport Departure Traffic Intensity": departures_airport_events,
            "Airport Arrival Traffic Intensity": arrivals_airport_events,
            "Total Airport Traffic Intensity": total_airport_events,
        }



In [ ]:
traffic_intensity_table = trajs_required.iterate_lazy().pipe(runway_movements).eval(max_workers=40, desc="Processing")

In [ ]:
final_table = final_table.merge(traffic_intensity_table, on="Flight ID", how="outer")

##### Determine Time Buffer betweeen succesive Landings (Exit Time - Entry Time)

In [ ]:
# Sort the final_table by "Entry Time" to ensure chronological order
final_table = final_table.sort_values(by="Entry Time").reset_index(drop=True)

# Retrieve the previous aircraft's exit time
final_table["Previous Exit Time"] = final_table["Exit Time"].shift(1)

# Calculate the time buffer between runway exit and the next entry
final_table["Time Reserve [s]"] = (final_table["Entry Time"] - final_table["Previous Exit Time"]).dt.total_seconds()

# Remove the intermediate column
final_table = final_table.drop(columns='Previous Exit Time')


#### Determine Traffic Information before Prediction Aircraft

##### Get all RWY Events on RWY14 (Approach and Go-Around)

In [ ]:
table_rwy_14 = table[table['Runway'] == '14']
table_rwy_14 = table_rwy_14.reset_index(drop=True)

##### Define Reference Point for Distance Measurement

In [ ]:
threshold_32 = airports['LSZH'].runways.list[3]

##### Function to determine basic information about preceding traffic

In [ ]:
def determine_previous_traffic(flight):
    prediction_time = flight.stop
    entry_time = final_table['Entry Time'].loc[final_table['Flight ID'] == flight.flight_id].iloc[0]

    if prediction_time is None:
        return None
    
    previous_traffic_table = table_rwy_14[(table_rwy_14['Exit Time'] >= prediction_time) & (table_rwy_14['Exit Time'] < entry_time)]
    number = len(previous_traffic_table)

    if not previous_traffic_table.empty:
        flight_id = previous_traffic_table['Flight ID'].iloc[-1]

        ac_type = previous_traffic_table['A/C Type'].iloc[-1]
        icao_code = previous_traffic_table['ICAO Code'].iloc[-1]
        icao_aricraft_typ = previous_traffic_table['ICAO Aircraft Type'].iloc[-1]
        number_engines = previous_traffic_table['Number of Engines'].iloc[-1]
        propulsion_type = previous_traffic_table['Propulsion Type'].iloc[-1]
        malw = previous_traffic_table['MALW [kg]'].iloc[-1]
        iwtc = previous_traffic_table['ICAO Weight Turbulence Category'].iloc[-1]
        gate = previous_traffic_table['Gate Region'].iloc[-1]
        distance = flight.distance(threshold_32).data['distance'].iloc[-1]


        return {
            "Flight ID": flight.flight_id,
            "Number of preceding Traffic": number,
            "Preceding Traffic ICAO Code": icao_code,
            "Preceding Traffic A/C Type": ac_type,
            "Preceding Traffic ICAO Aircraft Type": icao_aricraft_typ,
            "Preceding Traffic Propulsion Type": propulsion_type,
            "Preceding Traffic Number of Engines": number_engines,
            "Preceding Traffic MALW [kg]": malw,
            "Preceding Traffic ICAO Weight Turbulence Category": iwtc,
            "Preceding Traffic Gate": gate,
            "Flight ID preceding Traffic": flight_id,
            "Prediction Time": prediction_time,
            "Entry Time": entry_time,
            "Distance": distance
    }

    return {
        "Flight ID": flight.flight_id,
        "Number of preceding Traffic": 0
    }

In [ ]:
preceding_traffic_info_table = trajs_required.iterate_lazy().pipe(determine_previous_traffic).eval(max_workers=40, desc="Processing")

##### Create Dataframe with required Flights and their distance to the reference point

In [ ]:
preceding_ids = preceding_traffic_info_table['Flight ID preceding Traffic'].dropna().unique()

In [ ]:
trajs_preceding_traffic = trajs_required_complete[preceding_ids]

In [ ]:
trajs_preceding_traffic.to_parquet('debug_trajs_preceding_traffic.parquet')

In [ ]:
trajs_preceding_traffic = trajs_preceding_traffic.distance(threshold_32).eval(max_workers=1, desc="Processing")

##### Function to determine the altitude, speed and distance of the preceding flight at the prediction time

In [ ]:
def preceding_traffic_distance(flight):
    table_row = preceding_traffic_info_table[preceding_traffic_info_table['Flight ID preceding Traffic'] == flight.flight_id]
    if not table_row.empty:
        prediction_time = table_row['Prediction Time'].iloc[0]
        distance_pred = table_row['Distance'].iloc[0]
        pre_prediction_point = flight.at(prediction_time)
        alt = pre_prediction_point['geoaltitude']
        speed = pre_prediction_point['groundspeed']
        distance = pre_prediction_point['distance']

        distance_difference = distance_pred-distance

        flight_id = table_row['Flight ID'].iloc[0]


        return {
            "Flight ID": flight_id,
            "Preceding Aircraft Geoaltitude": alt,
            "Preceding Aircraft Speed": speed,
            "Distance to Preceding Aircraft": distance_difference,
        }

In [ ]:
preceding_traffic_distance_table = trajs_preceding_traffic.iterate_lazy().pipe(preceding_traffic_distance).eval(max_workers=40, desc='Processing')

In [ ]:
del trajs_preceding_traffic

##### Add Data about the preceding flight to the final final_table

In [ ]:
complete_preceding_traffic_table = preceding_traffic_info_table.merge(preceding_traffic_distance_table, on='Flight ID', how='outer')
complete_preceding_traffic_table = complete_preceding_traffic_table.drop(columns=['Flight ID preceding Traffic', 'Prediction Time', 'Entry Time', 'Distance'])
final_table = final_table.merge(complete_preceding_traffic_table, on='Flight ID', how='outer')
final_table = final_table.sort_values('Time of Prediction')
final_table = final_table.reset_index(drop=True)
del complete_preceding_traffic_table
del preceding_traffic_info_table

#### Get Traffic Information of successiv traffic

In [ ]:
airborne_dataframe = trajs_required_airborne.data
del trajs_required_airborne

#### Save Variables for Debugging

In [ ]:
final_table.to_csv('debug_final_table.csv', index=False)

In [ ]:
table_rwy_14.to_csv('debug_table_rwy_14.csv', index=False)

In [ ]:
trajs_required.to_parquet('debug_trajs_required.parquet')

In [ ]:
airborne_dataframe.to_parquet('debug_airborne_dataframe.parquet')

In [ ]:
airborne_dataframe = pd.read_parquet('debug_airborne_dataframe.parquet')

##### Function to determine information about successive traffic

In [ ]:
def successive_traffic(flight):
    time_th = 10 # minutes
    time_distance = 6 # minutes
    flight_id = flight.flight_id
    prediction_time = final_table['Time of Prediction'][final_table['Flight ID'] == flight_id].iloc[0]
    idx_flight = table_rwy_14.index[(table_rwy_14['Flight ID'] == flight_id) & (table_rwy_14['Event'] == 'Landing')].tolist()[0]
    upper_time_bound = prediction_time + pd.Timedelta(minutes=time_th)
    distance_time_bound = prediction_time + pd.Timedelta(minutes=time_distance)
    flight_data = flight.at(prediction_time)
    lat, lon = flight_data['latitude'], flight_data['longitude']

    dataframe = table_rwy_14[(table_rwy_14['Entry Time'] < upper_time_bound)]
    dataframe = dataframe.loc[idx_flight+1:]
    number = len(dataframe)
    if number > 0:
        dataframe = dataframe[(dataframe['Entry Time'] < distance_time_bound)]
        if not dataframe.empty:
            traffic_id = dataframe['Flight ID'].iloc[0]
            traffic_ac = dataframe['A/C Type'].iloc[0]

            traffic_data = airborne_dataframe[airborne_dataframe['timestamp'] == prediction_time]
            traffic_data = traffic_data[traffic_data['flight_id'] == traffic_id]
            if not traffic_data.empty:
                traffic_altitude = traffic_data['geoaltitude'].iloc[0]
                traffic_speed = traffic_data['groundspeed'].iloc[0]
                trafic_track = traffic_data['track'].iloc[0]
                traffic_vert_rate = traffic_data['vertical_rate'].iloc[0]
                

                lat_traffic, lon_traffic = traffic_data['latitude'].iloc[0], traffic_data['longitude'].iloc[0]

                coord_pred = (lat, lon)
                coord_traf = (lat_traffic, lon_traffic)
                distance = geodesic(coord_pred, coord_traf).nm


                return {
                    "Flight ID": flight_id,
                    "Number of Successive Traffic": number,
                    "Successive Traffic ID": traffic_id,
                    "Successive Traffic A/C Type": traffic_ac,
                    "Successive Traffic Geoaltitude": traffic_altitude,
                    "Successive Traffic Groundspeed": traffic_speed,
                    "Successive Traffic Track": trafic_track,
                    "Successive Traffic Vertical Rate": traffic_vert_rate,
                    "Successive Traffic Distance": distance
                }
            else:
                return {
                    "Flight ID": flight_id,
                    "Number of Successive Traffic": number
                }

    else:
        return {
            "Flight ID": flight_id,
            "Number of Successive Traffic": 0
        }


In [ ]:
successive_traffic_table = trajs_required.iterate_lazy().pipe(successive_traffic).eval(max_workers=10, desc="Processing")

##### Add successive Flight Information to Final Table

In [ ]:
final_table = final_table.merge(successive_traffic_table, on='Flight ID',how='outer')
final_table = final_table.sort_values('Time of Prediction')
final_table = final_table.reset_index(drop=True)

##### Encode Successive Traffic Track

In [ ]:
# Convert track angle to radians
track_rad = np.deg2rad(final_table['Successive Traffic Track'])

# Calculate sine & cosine and scale to range [0, 1]
final_table['Successive Traffic Track sin'] = (np.sin(track_rad) + 1) / 2
final_table['Successive Traffic Track cos'] = (np.cos(track_rad) + 1) / 2

# If no successive traffic exists → set sin & cos to 0
final_table.loc[final_table['Number of preceding Traffic'] == 0, ['Successive Traffic Track sin', 'Successive Traffic Track cos']] = 0


##### Successor Aligned

In [ ]:
rwy_heading = airports['LSZH'].runways.data['bearing'].loc[2]  # or dynamically from a column

# Check alignment with runway heading (using cyclic correction for 0/360 wraparound)
def is_aligned(track, heading, tolerance=5):
    if pd.isna(track):
        return False
    diff = abs((track - heading + 180) % 360 - 180)
    return diff <= tolerance  # return True if within tolerance

# Create new column with alignment result
final_table['Successor Aligned'] = final_table['Successive Traffic Track'].apply(lambda x: is_aligned(x, rwy_heading))


In [ ]:
final_table.head(10)

#### Determine ROT Trends

##### Remove Extreme ROTs

In [ ]:
final_table = final_table[final_table['ROT [s]'] < 150]
final_table = final_table[final_table['ROT [s]'] > 36]

In [ ]:
# Define new columns for the calculated metrics
columns = [
    'ROT Previous Flight [s]',
    'Average ROT Previous 5 Flight [s]',
    'Average ROT Previous Flight (Intensity) [s]',
    'ROT Previous Flight same A/C Type [s]',
    'Average ROT Previous 5 Flights same A/C Type [s]',
    'Average ROT Previous Flight same A/C Type (Intensity) [s]',
    'Average ROT Previous Flight same A/C Type Total [s]',
    'ROT Previous Flight same ICAO Weight Category [s]',
    'Average ROT Previous 5 Flights same ICAO Weight Category [s]',
    'Average ROT Previous Flight same ICAO Weight Category (Intensity) [s]',
    'Average ROT Previous Flight same ICAO Weight Category Total [s]',
    'Average ROT Total [s]'
]
final_table[columns] = None

# Iterate through each row in the final_table
for i, row in final_table.iterrows():
    flight_id = row['Flight ID']
    time_of_prediction = row['Time of Prediction']
    ac_type = row['A/C Type']
    weight_category = row['ICAO Weight Turbulence Category']
    
    # Filter past flights with Exit Time before the current prediction time
    relevant_flights = table_rwy_14[
        (table_rwy_14['Flight ID'] != flight_id) &
        (table_rwy_14['Exit Time'] < time_of_prediction)
    ].sort_values(by='Exit Time', ascending=False)
    
    # ROT of the most recent flight
    if not relevant_flights.empty:
        final_table.at[i, 'ROT Previous Flight [s]'] = relevant_flights.iloc[0]['ROT [s]']
    
    # Average ROT of the last 5 flights
    valid_rot_values = relevant_flights['ROT [s]'].iloc[:5].tolist()
    if valid_rot_values:
        final_table.at[i, 'Average ROT Previous 5 Flight [s]'] = sum(valid_rot_values) / len(valid_rot_values)
    
    # Average ROT of flights in the last 15 minutes (intensity)
    recent_flights = relevant_flights[relevant_flights['Exit Time'] >= (time_of_prediction - pd.Timedelta(minutes=15))]
    valid_rot_values_recent = recent_flights['ROT [s]'].tolist()
    if valid_rot_values_recent:
        final_table.at[i, 'Average ROT Previous Flight (Intensity) [s]'] = sum(valid_rot_values_recent) / len(valid_rot_values_recent)
    
    # --- Same A/C Type ---
    relevant_same_ac_type = relevant_flights[relevant_flights['A/C Type'] == ac_type]
    
    # ROT of the most recent flight with the same A/C type
    if not relevant_same_ac_type.empty:
        final_table.at[i, 'ROT Previous Flight same A/C Type [s]'] = relevant_same_ac_type.iloc[0]['ROT [s]']
    
    # Average ROT of the last 5 flights with the same A/C type
    valid_rot_values_same_ac_type = relevant_same_ac_type['ROT [s]'].iloc[:5].tolist()
    if valid_rot_values_same_ac_type:
        final_table.at[i, 'Average ROT Previous 5 Flights same A/C Type [s]'] = sum(valid_rot_values_same_ac_type) / len(valid_rot_values_same_ac_type)
    
    # Average ROT of same A/C type in the last 15 minutes
    recent_same_ac_type = relevant_same_ac_type[relevant_same_ac_type['Exit Time'] >= (time_of_prediction - pd.Timedelta(minutes=15))]
    valid_rot_values_intensity_same_ac_type = recent_same_ac_type['ROT [s]'].tolist()
    if valid_rot_values_intensity_same_ac_type:
        final_table.at[i, 'Average ROT Previous Flight same A/C Type (Intensity) [s]'] = sum(valid_rot_values_intensity_same_ac_type) / len(valid_rot_values_intensity_same_ac_type)
    
    # Average ROT of all past flights with the same A/C type
    valid_rot_values_total_same_ac_type = relevant_same_ac_type['ROT [s]'].tolist()
    if valid_rot_values_total_same_ac_type:
        final_table.at[i, 'Average ROT Previous Flight same A/C Type Total [s]'] = sum(valid_rot_values_total_same_ac_type) / len(valid_rot_values_total_same_ac_type)
    
    # --- Same ICAO Weight Category ---
    relevant_same_weight_category = relevant_flights[relevant_flights['ICAO Weight Turbulence Category'] == weight_category]
    
    # ROT of the most recent flight with the same weight category
    if not relevant_same_weight_category.empty:
        final_table.at[i, 'ROT Previous Flight same ICAO Weight Category [s]'] = relevant_same_weight_category.iloc[0]['ROT [s]']
    
    # Average ROT of the last 5 flights with the same weight category
    valid_rot_values_same_weight_category = relevant_same_weight_category['ROT [s]'].iloc[:5].tolist()
    if valid_rot_values_same_weight_category:
        final_table.at[i, 'Average ROT Previous 5 Flights same ICAO Weight Category [s]'] = sum(valid_rot_values_same_weight_category) / len(valid_rot_values_same_weight_category)
    
    # Average ROT of flights with the same weight category in the last 15 minutes
    recent_same_weight_category = relevant_same_weight_category[relevant_same_weight_category['Exit Time'] >= (time_of_prediction - pd.Timedelta(minutes=15))]
    valid_rot_values_intensity_same_weight_category = recent_same_weight_category['ROT [s]'].tolist()
    if valid_rot_values_intensity_same_weight_category:
        final_table.at[i, 'Average ROT Previous Flight same ICAO Weight Category (Intensity) [s]'] = sum(valid_rot_values_intensity_same_weight_category) / len(valid_rot_values_intensity_same_weight_category)
    
    # Average ROT of all past flights with the same weight category
    valid_rot_values_total_same_weight_category = relevant_same_weight_category['ROT [s]'].tolist()
    if valid_rot_values_total_same_weight_category:
        final_table.at[i, 'Average ROT Previous Flight same ICAO Weight Category Total [s]'] = sum(valid_rot_values_total_same_weight_category) / len(valid_rot_values_total_same_weight_category)

    # Overall average ROT of all previous flights (total)
    valid_rot_values_total = relevant_flights['ROT [s]'].tolist()
    if valid_rot_values_total:
        final_table.at[i, 'Average ROT Total [s]'] = sum(valid_rot_values_total) / len(valid_rot_values_total)


In [ ]:
final_table = final_table.sort_values('Time of Prediction').reset_index(drop=True)

In [ ]:
final_table.to_csv('Final_Table_raw.csv', index=False)
table.to_csv('Table.csv', index=False)

#### Remove unrealistic values

In [ ]:
final_table.loc[final_table['Successive Traffic Geoaltitude'] > 15000, 'Successive Traffic Geoaltitude'] = 15000

#### NaN Treatment

##### Weather Features

In [ ]:
# Fill missing wind direction values with 0
final_table['Wind direction [°]'] = final_table['Wind direction [°]'].fillna(0)

# Fill missing headwind values with total wind speed
final_table['Headwind [kt]'] = final_table['Headwind [kt]'].fillna(final_table['Wind speed [kt]'])

# Fill missing crosswind values with total wind speed
final_table['Crosswind [kt]'] = final_table['Crosswind [kt]'].fillna(final_table['Wind speed [kt]'])


##### Preceding Traffic Features

In [ ]:
final_table['Preceding Traffic A/C Type'] = final_table['Preceding Traffic A/C Type'].fillna('None')
final_table['Preceding Traffic ICAO Code'] = final_table['Preceding Traffic ICAO Code'].fillna('None')
final_table['Preceding Traffic ICAO Aircraft Type'] = final_table['Preceding Traffic ICAO Aircraft Type'].fillna('None')
final_table['Preceding Traffic Propulsion Type'] = final_table['Preceding Traffic Propulsion Type'].fillna('None')
final_table['Preceding Traffic Number of Engines'] = final_table['Preceding Traffic Number of Engines'].fillna(0)
final_table['Preceding Traffic MALW [kg]'] = final_table['Preceding Traffic MALW [kg]'].fillna(0)
final_table['Preceding Traffic ICAO Weight Turbulence Category'] = final_table['Preceding Traffic ICAO Weight Turbulence Category'].fillna('None')
final_table['Preceding Traffic Gate'] = final_table['Preceding Traffic Gate'].fillna('None')

final_table['Preceding Aircraft Geoaltitude'] = final_table['Preceding Aircraft Geoaltitude'].fillna(final_table['Preceding Aircraft Geoaltitude'].min()-25)
final_table['Preceding Aircraft Speed'] = final_table['Preceding Aircraft Speed'].fillna(0)
final_table['Distance to Preceding Aircraft'] = final_table['Distance to Preceding Aircraft'].fillna(final_table['Distance to Preceding Aircraft'].max()+1)

##### Successive Traffic Features

In [ ]:
final_table['Number of Successive Traffic'] = final_table['Number of Successive Traffic'].fillna(0)
final_table['Successive Traffic ID'] = final_table['Successive Traffic ID'].fillna('None')
final_table['Successive Traffic A/C Type'] = final_table['Successive Traffic A/C Type'].fillna('None')
final_table['Successive Traffic Geoaltitude'] = final_table['Successive Traffic Geoaltitude'].fillna(final_table['Successive Traffic Geoaltitude'].max()+1000)
final_table['Successive Traffic Groundspeed'] = final_table['Successive Traffic Groundspeed'].fillna(final_table['Successive Traffic Groundspeed'].max()+10)
final_table['Successive Traffic Track'] = final_table['Successive Traffic Track'].fillna(rwy_heading+180)
final_table['Successive Traffic Vertical Rate'] = final_table['Successive Traffic Vertical Rate'].fillna(0)
final_table['Successive Traffic Distance'] = final_table['Successive Traffic Distance'].fillna(final_table['Successive Traffic Distance'].max()+1)
final_table['Successive Traffic Track sin'] = final_table['Successive Traffic Track sin'].fillna(0)
final_table['Successive Traffic Track cos'] = final_table['Successive Traffic Track cos'].fillna(0)

##### ROT Trend Features

In [ ]:
final_table['Average ROT Previous Flight same ICAO Weight Category Total [s]'] = final_table.apply(
    lambda row: row['Average ROT Total [s]']
    if pd.isna(row['Average ROT Previous Flight same ICAO Weight Category Total [s]'])
    else row['Average ROT Previous Flight same ICAO Weight Category Total [s]'],
    axis=1
)


In [ ]:
final_table['Average ROT Previous Flight same ICAO Weight Category (Intensity) [s]'] = final_table.apply(
    lambda row: row['Average ROT Previous Flight same ICAO Weight Category Total [s]']
    if pd.isna(row['Average ROT Previous Flight same ICAO Weight Category (Intensity) [s]'])
    else row['Average ROT Previous Flight same ICAO Weight Category (Intensity) [s]'],
    axis=1
)


In [ ]:
final_table['Average ROT Previous 5 Flights same ICAO Weight Category [s]'] = final_table.apply(
    lambda row: row['Average ROT Total [s]']
    if pd.isna(row['Average ROT Previous 5 Flights same ICAO Weight Category [s]'])
    else row['Average ROT Previous 5 Flights same ICAO Weight Category [s]'],
    axis=1
)


In [ ]:
final_table['ROT Previous Flight same ICAO Weight Category [s]'] = final_table.apply(
    lambda row: row['Average ROT Total [s]']
    if pd.isna(row['ROT Previous Flight same ICAO Weight Category [s]'])
    else row['ROT Previous Flight same ICAO Weight Category [s]'],
    axis=1
)


In [ ]:
final_table['Average ROT Previous Flight same A/C Type Total [s]'] = final_table.apply(
    lambda row: row['Average ROT Previous Flight same ICAO Weight Category Total [s]']
    if pd.isna(row['Average ROT Previous Flight same A/C Type Total [s]'])
    else row['Average ROT Previous Flight same A/C Type Total [s]'],
    axis=1
)


In [ ]:
final_table['Average ROT Previous Flight same A/C Type (Intensity) [s]'] = final_table.apply(
    lambda row: row['Average ROT Previous Flight same ICAO Weight Category (Intensity) [s]']
    if pd.isna(row['Average ROT Previous Flight same A/C Type (Intensity) [s]'])
    else row['Average ROT Previous Flight same A/C Type (Intensity) [s]'],
    axis=1
)


In [ ]:
final_table['Average ROT Previous 5 Flights same A/C Type [s]'] = final_table.apply(
    lambda row: row['Average ROT Previous 5 Flights same ICAO Weight Category [s]']
    if pd.isna(row['Average ROT Previous 5 Flights same A/C Type [s]'])
    else row['Average ROT Previous 5 Flights same A/C Type [s]'],
    axis=1
)


In [ ]:
final_table['ROT Previous Flight same A/C Type [s]'] = final_table.apply(
    lambda row: row['ROT Previous Flight same ICAO Weight Category [s]']
    if pd.isna(row['ROT Previous Flight same A/C Type [s]'])
    else row['ROT Previous Flight same A/C Type [s]'],
    axis=1
)


In [ ]:
final_table['Average ROT Previous Flight (Intensity) [s]'] = final_table.apply(
    lambda row: row['Average ROT Total [s]']
    if pd.isna(row['Average ROT Previous Flight (Intensity) [s]'])
    else row['Average ROT Previous Flight (Intensity) [s]'],
    axis=1
)


In [ ]:
final_table

In [ ]:
plt.hist(final_table['ROT [s]'], bins=range(30,151), edgecolor='black')

In [ ]:
final_table.to_csv('Final_Table.csv', index=False)